# 7. Input lphafold assessment

#### Context:
In addition to our protein annots, we've structurally modelled ~1,200 GORG Dark rhodopsins in alphafold3<br>
And want to know which of these have an 8th, N-terminal helix.<Br>
Since that seems to be a new characteristic of Ferredoxin-associated rhodopsins (FeRh a.k.a. 2Fe2S rhodopsins)

#### Goal:
Narrow down the rhodopsins that I need to inspect for an N-terminal 8th helix<br>
(Since I cant realistically look at ~1,200 alphafold models in PAE viewer)<br>

#### Approach:
To group the rhodopsins in a sequence aligner (Jalview) into similar types by AA similarity.<br>
So that I can examine 1 representative of each type.<br>

#### Prior to this:
Tianyi aligned all 1,200 rhodopsins in Jalview.<br>
Maria generated a rhodopsin phylogeny.<br>
https://itol.embl.de/tree/12812822615690281732644278#<br>
(That informs which rhods we consider to be in the 2Fe2S rhodopsin clade).<br>

#### Notebook Organization:
This is an iterative, semi-manual process that has several programmatic steps and several manual ones.<br>
I have broken the notebook into code blocks called:<br>
* Parts I, II, III, IV
* Where a manual step happened between each part.<br> (E.g. visual assessment of a sequence alignment or visual assessment of alphafold models).<br>

#### Data Products:
1. Updates the iToL table so that we can plot presence/absence of 8th helix against the phylogeny.
2. Of 2Fe2S rhodopsins, figures out which have the been truncated at the N'terminus. (I.e. the 8th helix is there, but incomplete).
3. Makes a list of full-length 2Fe2S rhodopsins bearing the 8th helix (so that Tianyi can re-align AAs and make an HMM profile for the 8th helix.)
4. Updates the rhodopsin metadata table to reflect
    <br>A. 8th helix presence/absence/incompleteness
    <br>B. Subclade info (whether 2Fe2Srhodopsin or FArhodopsin)


### Part I.
Subsetting rhodopsin alignment so that I can evaluate 2Fe2S rhodopsins and non2Fe2S rhodopsins separately in a sequence aligner GUI.<br>
**Why?**<Br>
It will be easier to tell which 2Fe2S type rhodopsins are complete, if they have their own alignment.<br>
(This excludes outgroups that could have weird or long N termini)


In [1]:
import re
from Bio import SeqIO

#### INPUTS #####
# Alignment with ALL ~1200 GORG-Dark rhodopsins
PATH_input_1 = "./0_input_rhod_alignment_by_Tianyi/aligned_sequences.fasta"

#### List of promising rhodopsin IDs based on a preliminary review of the above file
LIST_IDs_alignable_8th_helix=["AM-388-N16 contigs AM-388-N16 NODE 36 15", "AM-388-D08 contigs AM-388-D08 NODE 84 2", "AH-985-B07 contigs AH-985-B07 NODE 9 10", "AH-470-E07 contigs AH-470-E07 NODE 2 54", "AM-276-K05 contigs AM-276-K05 NODE 11 24", "AM-332-O14 contigs AM-332-O14 NODE 1 129", "AM-247-G17 contigs AM-247-G17 NODE 32 19", "AH-473-N09 contigs AH-473-N09 NODE 54 5", "AM-332-C16 contigs AM-332-C16 NODE 8 7", "AM-332-M03 contigs AM-332-M03 NODE 38 4", "AM-332-E21 contigs AM-332-E21 NODE 15 18", "AM-332-J04 contigs AM-332-J04 NODE 8 9", "AM-332-N05 contigs AM-332-N05 NODE 2 6", "AH-985-D04 contigs AH-985-D04 NODE 22 2", "AM-332-F20 contigs AM-332-F20 NODE 43 5", "AM-332-L13 contigs AM-332-L13 NODE 52 5", "AM-288-O17 contigs AM-288-O17 NODE 6 28", "AM-332-C03 contigs AM-332-C03 NODE 18 4", "AM-332-K10 contigs AM-332-K10 NODE 9 22", "AM-332-K13 contigs AM-332-K13 NODE 2 2", "AH-470-L18 contigs AH-470-L18 NODE 26 5", "AH-982-J09 contigs AH-982-J09 NODE 10 7", "AH-982-L18 contigs AH-982-L18 NODE 12 4", "AH-985-E13 contigs AH-985-E13 NODE 1 133", "AH-985-J19 contigs AH-985-J19 NODE 2 49", "AM-332-E15 contigs AM-332-E15 NODE 10 5", "AM-276-P03 contigs AM-276-P03 NODE 2 149", "AM-332-L06 contigs AM-332-L06 NODE 14 2", "AM-332-C14 contigs AM-332-C14 NODE 11 9", "AM-332-P08 contigs AM-332-P08 NODE 42 3", "AM-332-K08 contigs AM-332-K08 NODE 18 22", "AM-332-E05 contigs AM-332-E05 NODE 4 106", "AM-685-E19 contigs SCGC AM-685-E19 contig6 29", "AH-302-L09 contigs AH-302-L09 NODE 11 3", "AH-305-A06 contigs AH-305-A06 NODE 1 19", "AH-305-M13 contigs AH-305-M13 NODE 9 6", "AH-307-I04 contigs AH-307-I04 NODE 6 11", "AG-538-G09 contigs AG-538-G09 NODE 17 4", "AH-309-A13 contigs AH-309-A13 NODE 23 14", "AM-124-I15 contigs AM-124-I15 NODE 53 2", "AM-276-A01 contigs AM-276-A01 NODE 21 6", "AM-229-K19 contigs AM-229-K19 NODE 37 2", "AM-124-E05 contigs AM-124-E05 NODE 5 31", "AM-161-G23 contigs AM-161-G23 NODE 14 19", "AM-276-F23 contigs AM-276-F23 NODE 5 6", "AM-127-K13 contigs AM-127-K13 NODE 40 6", "AM-243-J14 contigs AM-243-J14 NODE 4 6", "AM-249-N20 contigs AM-249-N20 NODE 7 21", "AH-573-D18 contigs AH-573-D18 NODE 1 7", "AH-307-G21 contigs AH-307-G21 NODE 16 15", "AM-210-D16 contigs AM-210-D16 NODE 2 12", "AM-124-C13 contigs AM-124-C13 NODE 1 53", "AM-135-L13 contigs AM-135-L13 NODE 3 34", "AM-184-E04 contigs AM-184-E04 NODE 2 34", "AM-276-A04 contigs AM-276-A04 NODE 3 8", "AM-280-L17 contigs AM-280-L17 NODE 1 29", "AM-161-A19 contigs AM-161-A19 NODE 39 1", "AM-243-K07 contigs AM-243-K07 NODE 45 6", "AM-284-B07 contigs AM-284-B07 NODE 1 42", "AM-158-K11 contigs AM-158-K11 NODE 35 6", "AH-305-K23 contigs AH-305-K23 NODE 9 11", "AM-255-O11 contigs AM-255-O11 NODE 2 57", "AM-118-P09 contigs AM-118-P09 NODE 6 25", "AM-334-O09 contigs AM-334-O09 NODE 4 24", "AM-124-G09 contigs AM-124-G09 NODE 36 11", "AM-280-G11 contigs AM-280-G11 NODE 31 6", "AM-249-M11 contigs AM-249-M11 NODE 6 11", "AM-288-F06 contigs AM-288-F06 NODE 16 8", "AM-255-C18 contigs AM-255-C18 NODE 18 7", "AH-573-A09 contigs AH-573-A09 NODE 10 29", "AH-573-O15 contigs AH-573-O15 NODE 7 18", "AM-184-P16 contigs AM-184-P16 NODE 22 5", "AM-264-F14 contigs AM-264-F14 NODE 42 3", "AH-988-B10 contigs AH-988-B10 NODE 8 10", "AM-124-F21 contigs AM-124-F21 NODE 37 3", "AM-135-M08 contigs AM-135-M08 NODE 13 12", "AM-121-N16 contigs AM-121-N16 NODE 23 6", "AM-280-K21 contigs AM-280-K21 NODE 13 19", "AM-118-K03 contigs AM-118-K03 NODE 4 20", "AM-229-F19 contigs AM-229-F19 NODE 1 60", "AM-121-I21 contigs AM-121-I21 NODE 2 56", "AM-208-C11 contigs AM-208-C11 NODE 8 13", "AM-184-L15 contigs AM-184-L15 NODE 1 56", "AM-187-O05 contigs AM-187-O05 NODE 6 18", "AM-118-K21 contigs AM-118-K21 NODE 29 2", "AM-161-C03 contigs AM-161-C03 NODE 5 16", "AM-184-I06 contigs AM-184-I06 NODE 7 1", "AM-121-P05 contigs AM-121-P05 NODE 2 82", "AM-255-M09 contigs AM-255-M09 NODE 6 19", "AH-573-M15 contigs AH-573-M15 NODE 20 5", "AH-988-C21 contigs AH-988-C21 NODE 8 18", "AM-208-K18 contigs AM-208-K18 NODE 9 14", "AM-210-J07 contigs AM-210-J07 NODE 1 73", "AM-187-J02 contigs AM-187-J02 NODE 11 11", "AM-249-B14 contigs AM-249-B14 NODE 11 9", "AM-332-C08 contigs AM-332-C08 NODE 8 13", "AM-243-C16 contigs AM-243-C16 NODE 23 6", "AM-288-B14 contigs AM-288-B14 NODE 1 73", "AM-118-K20 contigs AM-118-K20 NODE 12 12", "AM-264-E23 contigs AM-264-E23 NODE 9 12", "AM-121-E03 contigs AM-121-E03 NODE 32 7", "AM-280-C14 contigs AM-280-C14 NODE 12 13", "AM-334-G07 contigs AM-334-G07 NODE 8 4", "AM-334-F08 contigs AM-334-F08 NODE 16 12", "AM-249-M09 contigs AM-249-M09 NODE 76 3", "AM-118-G13 contigs AM-118-G13 NODE 18 8", "AM-184-M06 contigs AM-184-M06 NODE 3 16", "AG-538-D17 contigs AG-538-D17 NODE 27 3", "AM-118-P17 contigs AM-118-P17 NODE 3 7", "AM-249-K22 contigs AM-249-K22 NODE 1 30", "AM-262-E15 contigs AM-262-E15 NODE 12 11", "AM-332-G04 contigs AM-332-G04 NODE 9 17", "AM-216-B07 contigs AM-216-B07 NODE 17 7", "AH-573-P04 contigs AH-573-P04 NODE 2 36", "AM-187-C18 contigs AM-187-C18 NODE 24 8", "AH-611-A20 contigs AH-611-A20 NODE 13 33", "AM-216-K15 contigs AM-216-K15 NODE 2 34", "AM-334-D04 contigs AM-334-D04 NODE 17 6", "AM-288-P10 contigs AM-288-P10 NODE 5 23", "AH-562-L13 contigs AH-562-L13 NODE 4 53", "AM-208-K04 contigs AM-208-K04 NODE 11 1", "AH-305-K03 contigs AH-305-K03 NODE 1 39", "AM-216-D10 contigs AM-216-D10 NODE 3 6", "AH-735-B03 contigs AH-735-B03 NODE 1 75", "AM-216-C21 contigs AM-216-C21 NODE 24 5", "AM-225-L16 contigs AM-225-L16 NODE 28 3", "AM-216-J22 contigs AM-216-J22 NODE 18 5", "AM-208-C02 contigs AM-208-C02 NODE 49 1", "AM-262-I09 contigs AM-262-I09 NODE 1 1", "AH-573-C21 contigs AH-573-C21 NODE 17 3", "AH-573-F17 contigs AH-573-F17 NODE 1 133", "AH-735-G09 contigs AH-735-G09 NODE 22 8", "AH-888-M17 contigs AH-888-M17 NODE 10 4", "AM-286-I22 contigs AM-286-I22 NODE 19 7", "AH-735-J03 contigs AH-735-J03 NODE 15 22", "AM-121-O09 contigs AM-121-O09 NODE 22 3", "AM-164-A01 contigs AM-164-A01 NODE 7 4", "AM-187-K07 contigs AM-187-K07 NODE 20 8", "AH-985-K08 contigs AH-985-K08 NODE 4 21", "AM-208-D10 contigs AM-208-D10 NODE 22 11", "AM-118-M15 contigs AM-118-M15 NODE 7 18", "AM-235-I09 contigs AM-235-I09 NODE 20 3", "AM-247-A03 contigs AM-247-A03 NODE 6 10", "AM-249-G22 contigs AM-249-G22 NODE 9 1", "AM-235-K05 contigs AM-235-K05 NODE 26 4", "AM-249-I18 contigs AM-249-I18 NODE 25 8", "AM-286-O15 contigs AM-286-O15 NODE 9 11", "AM-138-C10 contigs AM-138-C10 NODE 30 4", "AH-573-A18 contigs AH-573-A18 NODE 23 7", "AH-613-J09 contigs AH-613-J09 NODE 12 21", "AH-888-G18 contigs AH-888-G18 NODE 18 8", "AH-735-N06 contigs AH-735-N06 NODE 20 7", "AM-332-I06 contigs AM-332-I06 NODE 5 8", "AH-991-G23 contigs AH-991-G23 NODE 11 2", "AM-255-N03 contigs AM-255-N03 NODE 25 9", "AM-201-C21 contigs AM-201-C21 NODE 36 6", "AM-118-M18 contigs AM-118-M18 NODE 11 14", "AM-255-M18 contigs AM-255-M18 NODE 31 6", "AH-611-F04 contigs AH-611-F04 NODE 13 7", "AH-473-B13 contigs AH-473-B13 NODE 22 8", "AM-187-E15 contigs AM-187-E15 NODE 11 21", "AM-216-C16 contigs AM-216-C16 NODE 7 20", "AH-735-P04 contigs AH-735-P04 NODE 60 1", "AH-991-F05 contigs AH-991-F05 NODE 29 4", "AM-201-G09 contigs AM-201-G09 NODE 8 11", "AM-255-P07 contigs AM-255-P07 NODE 53 5", "AM-264-L18 contigs AM-264-L18 NODE 9 7", "AM-164-B09 contigs AM-164-B09 NODE 24 6", "AM-201-G23 contigs AM-201-G23 NODE 4 9", "AH-302-C16 contigs AH-302-C16 NODE 2 56", "AH-988-D21 contigs AH-988-D21 NODE 11 8", "AM-158-K17 contigs AM-158-K17 NODE 3 8", "AH-611-L14 contigs AH-611-L14 NODE 5 3", "AH-735-A01 contigs AH-735-A01 NODE 12 8", "AM-118-A20 contigs AM-118-A20 NODE 46 1", "AM-280-E05 contigs AM-280-E05 NODE 9 7", "AH-735-M18 contigs AH-735-M18 NODE 1 10", "AH-573-N23 contigs AH-573-N23 NODE 12 8", "AM-685-G04 contigs SCGC AM-685-G04 contig7 1", "AM-235-J07 contigs AM-235-J07 NODE 9 13", "AH-888-O18 contigs AH-888-O18 NODE 13 3", "AH-611-D02 contigs AH-611-D02 NODE 22 5", "AM-262-N05 contigs AM-262-N05 NODE 6 1", "AH-888-D11 contigs AH-888-D11 NODE 14 1", "AM-247-K15 contigs AM-247-K15 NODE 5 6", "AM-334-B22 contigs AM-334-B22 NODE 3 22", "AM-138-J05 contigs AM-138-J05 NODE 14 7", "AM-201-O16 contigs AM-201-O16 NODE 13 10", "AM-138-J05 contigs AM-138-J05 NODE 14 13", "AH-735-E08 contigs AH-735-E08 NODE 3 9", "AH-613-I15 contigs AH-613-I15 NODE 8 11", "AM-288-J07 contigs AM-288-J07 NODE 3 47", "AM-286-B19 contigs AM-286-B19 NODE 9 12", "AH-735-G22 contigs AH-735-G22 NODE 9 10", "AM-334-O18 contigs AM-334-O18 NODE 36 6", "AG-538-B02 contigs AG-538-B02 NODE 14 4", "AM-118-J17 contigs AM-118-J17 NODE 11 23", "AH-735-M08 contigs AH-735-M08 NODE 19 10", "AH-735-E21 contigs AH-735-E21 NODE 15 1", "AM-208-L13 contigs AM-208-L13 NODE 13 10", "AH-305-P08 contigs AH-305-P08 NODE 2 27", "AM-259-G09 contigs AM-259-G09 NODE 1 27", "AM-276-M20 contigs AM-276-M20 NODE 23 17", "AM-280-J04 contigs AM-280-J04 NODE 36 1", "AM-127-L08 contigs AM-127-L08 NODE 54 6", "AM-249-I23 contigs AM-249-I23 NODE 4 13", "AM-276-N02 contigs AM-276-N02 NODE 3 36", "AH-302-P11 contigs AH-302-P11 NODE 6 24", "AM-235-B08 contigs AM-235-B08 NODE 6 15", "AM-288-P17 contigs AM-288-P17 NODE 2 39", "AM-332-L17 contigs AM-332-L17 NODE 8 10", "AM-334-C06 contigs AM-334-C06 NODE 11 9", "AM-332-N13 contigs AM-332-N13 NODE 3 11", "AM-332-A16 contigs AM-332-A16 NODE 16 2", "AM-332-B10 contigs AM-332-B10 NODE 8 9", "AH-735-N13 contigs AH-735-N13 NODE 29 5", "AH-888-L04 contigs AH-888-L04 NODE 15 6", "AM-247-J07 contigs AM-247-J07 NODE 30 6", "AM-332-G10 contigs AM-332-G10 NODE 17 14", "AH-305-D13 contigs AH-305-D13 NODE 54 1", "AM-130-K09 contigs AM-130-K09 NODE 64 1", "AH-470-P15 contigs AH-470-P15 NODE 3 1", "AM-332-P04 contigs AM-332-P04 NODE 9 9", "AM-332-C06 contigs AM-332-C06 NODE 4 4", "AM-332-F18 contigs AM-332-F18 NODE 2 10", "AH-985-A17 contigs AH-985-A17 NODE 1 74", "AH-991-E06 contigs AH-991-E06 NODE 9 14", "AM-288-B03 contigs AM-288-B03 NODE 7 10", "AH-611-C05 contigs AH-611-C05 NODE 43 6", "AH-470-A20 contigs AH-470-A20 NODE 34 1", "AH-611-N23 contigs AH-611-N23 NODE 2 61", "AH-985-E07 contigs AH-985-E07 NODE 5 33", "AH-468-K22 contigs AH-468-K22 NODE 12 6", "AH-470-C17 contigs AH-470-C17 NODE 26 12", "AH-470-J19 contigs AH-470-J19 NODE 1 19", "AH-470-O19 contigs AH-470-O19 NODE 17 3", "AH-473-A19 contigs AH-473-A19 NODE 2 13", "AH-982-I21 contigs AH-982-I21 NODE 1 111", "AH-991-L03 contigs AH-991-L03 NODE 9 5", "AM-235-D23 contigs AM-235-D23 NODE 45 1", "AH-473-F04 contigs AH-473-F04 NODE 4 28", "AH-473-J16 contigs AH-473-J16 NODE 17 6", "AH-473-O05 contigs AH-473-O05 NODE 1 56", "AH-470-P11 contigs AH-470-P11 NODE 6 45", "AH-982-G22 contigs AH-982-G22 NODE 2 170", "AH-982-C03 contigs AH-982-C03 NODE 1 59", "AH-470-F08 contigs AH-470-F08 NODE 4 18", "AH-473-I07 contigs AH-473-I07 NODE 7 3", "AH-470-N15 contigs AH-470-N15 NODE 3 14", "AH-473-D09 contigs AH-473-D09 NODE 22 4", "AH-985-B18 contigs AH-985-B18 NODE 9 37", "AH-473-N19 contigs AH-473-N19 NODE 7 5", "AH-985-O05 contigs AH-985-O05 NODE 3 98", "AH-473-F18 contigs AH-473-F18 NODE 5 35", "AH-473-J14 contigs AH-473-J14 NODE 2 22", "AH-985-E22 contigs AH-985-E22 NODE 5 10", "AM-332-A22 contigs AM-332-A22 NODE 1 15", "AM-332-M16 contigs AM-332-M16 NODE 6 5", "AM-332-D02 contigs AM-332-D02 NODE 29 1", "AM-332-D03 contigs AM-332-D03 NODE 11 29", "AM-332-F21 contigs AM-332-F21 NODE 6 37", "AM-332-L08 contigs AM-332-L08 NODE 2 7", "AM-334-A01 contigs AM-334-A01 NODE 10 10", "AM-332-N08 contigs AM-332-N08 NODE 21 5", "AM-334-N11 contigs AM-334-N11 NODE 1 11", "AM-235-K23 contigs AM-235-K23 NODE 3 24", "AH-473-C07 contigs AH-473-C07 NODE 5 25", "AH-611-I23 contigs AH-611-I23 NODE 4 21", "AH-611-P07 contigs AH-611-P07 NODE 10 4", "AM-235-M17 contigs AM-235-M17 NODE 15 7", "AH-611-O09 contigs AH-611-O09 NODE 4 21", "AM-235-A09 contigs AM-235-A09 NODE 37 4", "AM-164-N21 contigs AM-164-N21 NODE 43 6", "AH-470-I05 contigs AH-470-I05 NODE 3 24", "AG-538-J23 contigs AG-538-J23 NODE 9 13", "AM-187-G11 contigs AM-187-G11 NODE 6 31", "AH-613-E11 contigs AH-613-E11 NODE 44 3", "AH-613-E20 contigs AH-613-E20 NODE 13 9", "AM-247-E21 contigs AM-247-E21 NODE 2 64", "AM-187-P09 contigs AM-187-P09 NODE 2 10", "AM-187-J20 contigs AM-187-J20 NODE 47 1", "AM-225-A20 contigs AM-225-A20 NODE 2 8", "AM-225-M07 contigs AM-225-M07 NODE 45 1", "AM-201-A14 contigs AM-201-A14 NODE 20 4", "AM-164-D11 contigs AM-164-D11 NODE 5 20", "AM-334-M02 contigs AM-334-M02 NODE 10 8", "AM-334-O04 contigs AM-334-O04 NODE 7 15", "AH-735-I22 contigs AH-735-I22 NODE 12 5", "AM-685-F13 contigs SCGC AM-685-F13 contig16 8", "AM-332-C20 contigs AM-332-C20 NODE 3 54", "AH-613-F09 contigs AH-613-F09 NODE 36 3", "AH-985-D18 contigs AH-985-D18 NODE 3 60", "AM-286-G07 contigs AM-286-G07 NODE 1 25", "AM-184-C05 contigs AM-184-C05 NODE 7 8", "AM-130-G06 contigs AM-130-G06 NODE 6 65", "AM-259-O02 contigs AM-259-O02 NODE 1 31", "AM-332-B18 contigs AM-332-B18 NODE 12 2", "AM-332-N20 contigs AM-332-N20 NODE 1 20", "AM-164-L02 contigs AM-164-L02 NODE 1 29", "AM-243-A07 contigs AM-243-A07 NODE 5 36", "AM-235-I03 contigs AM-235-I03 NODE 31 2", "AM-334-E18 contigs AM-334-E18 NODE 1 26", "AH-613-E07 contigs AH-613-E07 NODE 18 12", "AH-735-K15 contigs AH-735-K15 NODE 2 5", "AM-216-E08 contigs AM-216-E08 NODE 31 6", "AH-991-K03 contigs AH-991-K03 NODE 2 46", "AM-201-N19 contigs AM-201-N19 NODE 5 6", "AM-235-G21 contigs AM-235-G21 NODE 12 20", "AM-332-K06 contigs AM-332-K06 NODE 3 14", "AM-332-D08 contigs AM-332-D08 NODE 14 9", "AM-332-I16 contigs AM-332-I16 NODE 4 12", "AM-332-G22 contigs AM-332-G22 NODE 2 60", "AM-216-G06 contigs AM-216-G06 NODE 2 33", "AM-262-C09 contigs AM-262-C09 NODE 10 9", "AM-201-K09 contigs AM-201-K09 NODE 20 2", "AM-206-N08 contigs AM-206-N08 NODE 12 23", "AM-286-F07 contigs AM-286-F07 NODE 1 59", "AM-286-K06 contigs AM-286-K06 NODE 1 31", "AH-735-G21 contigs AH-735-G21 NODE 2 64", "AH-735-M23 contigs AH-735-M23 NODE 2 22", "AH-888-L22 contigs AH-888-L22 NODE 6 17", "AH-985-I18 contigs AH-985-I18 NODE 3 27", "AM-235-L08 contigs AM-235-L08 NODE 7 29", "AM-235-D14 contigs AM-235-D14 NODE 4 11", "AM-288-J17 contigs AM-288-J17 NODE 1 17", "AM-249-B18 contigs AM-249-B18 NODE 8 20", "AM-235-D15 contigs AM-235-D15 NODE 6 13", "AM-243-E13 contigs AM-243-E13 NODE 13 4", "AM-118-B02 contigs AM-118-B02 NODE 29 7", "AH-613-N10 contigs AH-613-N10 NODE 74 2", "AM-243-C08 contigs AM-243-C08 NODE 4 23", "AM-229-O19 contigs AM-229-O19 NODE 1 66", "AH-888-C20 contigs AH-888-C20 NODE 1 33", "AH-573-J17 contigs AH-573-J17 NODE 13 27", "AH-611-P06 contigs AH-611-P06 NODE 3 17", "AH-613-C13 contigs AH-613-C13 NODE 7 13", "AM-235-C09 contigs AM-235-C09 NODE 40 1", "AM-685-E15 contigs SCGC AM-685-E15 contig5 25", "AH-735-O16 contigs AH-735-O16 NODE 1 23", "AM-235-O15 contigs AM-235-O15 NODE 1 24", "AM-685-F04 contigs SCGC AM-685-F04 contig7 14", "AH-613-J18 contigs AH-613-J18 NODE 9 7", "AM-235-J04 contigs AM-235-J04 NODE 59 1", "AM-243-A06 contigs AM-243-A06 NODE 1 62", "AM-187-B23 contigs AM-187-B23 NODE 23 8", "AM-235-P18 contigs AM-235-P18 NODE 1 65", "AH-735-A03 contigs AH-735-A03 NODE 13 1", "AM-187-K08 contigs AM-187-K08 NODE 6 7", "AH-888-O06 contigs AH-888-O06 NODE 1 82", "AM-243-F23 contigs AM-243-F23 NODE 3 5", "AM-247-K09 contigs AM-247-K09 NODE 18 15", "AM-286-N18 contigs AM-286-N18 NODE 4 34", "AM-187-M17 contigs AM-187-M17 NODE 7 4", "AH-735-A10 contigs AH-735-A10 NODE 2 9", "AM-247-P05 contigs AM-247-P05 NODE 1 23", "AM-243-B08 contigs AM-243-B08 NODE 21 3", "AH-735-L16 contigs AH-735-L16 NODE 2 28", "AH-991-J22 contigs AH-991-J22 NODE 3 13", "AM-235-F14 contigs AM-235-F14 NODE 2 22", "AH-735-L20 contigs AH-735-L20 NODE 12 19", "AM-184-O13 contigs AM-184-O13 NODE 1 26", "AM-187-F11 contigs AM-187-F11 NODE 3 15", "AM-689-N03 contigs SCGC AM-689-N03 contig1 25", "AM-164-M18 contigs AM-164-M18 NODE 6 28", "AH-613-I09 contigs AH-613-I09 NODE 4 17", "AH-991-D11 contigs AH-991-D11 NODE 21 6", "AM-286-I03 contigs AM-286-I03 NODE 12 1", "AM-216-K08 contigs AM-216-K08 NODE 54 1", "AH-735-D20 contigs AH-735-D20 NODE 20 7", "AM-130-E23 contigs AM-130-E23 NODE 6 20", "AM-210-A17 contigs AM-210-A17 NODE 12 15", "AM-243-F04 contigs AM-243-F04 NODE 15 6", "AM-284-A18 contigs AM-284-A18 NODE 43 6", "AM-334-B19 contigs AM-334-B19 NODE 15 1", "AM-118-E14 contigs AM-118-E14 NODE 6 12", "AH-611-J16 contigs AH-611-J16 NODE 2 19", "AH-735-K19 contigs AH-735-K19 NODE 5 13", "AM-208-K11 contigs AM-208-K11 NODE 27 6", "AM-247-B14 contigs AM-247-B14 NODE 1 43", "AH-735-K11 contigs AH-735-K11 NODE 4 13", "AM-138-J06 contigs AM-138-J06 NODE 3 17", "AG-538-M09 contigs AG-538-M09 NODE 28 3", "AH-562-C07 contigs AH-562-C07 NODE 38 7", "AH-562-E04 contigs AH-562-E04 NODE 23 5", "AH-888-G20 contigs AH-888-G20 NODE 29 2", "AM-121-N18 contigs AM-121-N18 NODE 5 23", "AM-286-F05 contigs AM-286-F05 NODE 1 41", "AH-562-O14 contigs AH-562-O14 NODE 25 4", "AM-130-I22 contigs AM-130-I22 NODE 14 8", "AM-201-F22 contigs AM-201-F22 NODE 25 6", "AM-138-G09 contigs AM-138-G09 NODE 34 7", "AH-562-K21 contigs AH-562-K21 NODE 7 20", "AM-334-O03 contigs AM-334-O03 NODE 1 55", "AH-888-A19 contigs AH-888-A19 NODE 3 15", "AM-124-K09 contigs AM-124-K09 NODE 2 6", "AM-288-O07 contigs AM-288-O07 NODE 2 37", "AM-216-O05 contigs AM-216-O05 NODE 7 4", "AH-735-G19 contigs AH-735-G19 NODE 1 1", "AM-138-K04 contigs AM-138-K04 NODE 10 19", "AH-470-B04 contigs AH-470-B04 NODE 24 3", "AH-470-K09 contigs AH-470-K09 NODE 24 5", "AH-985-C23 contigs AH-985-C23 NODE 7 17", "AH-473-M06 contigs AH-473-M06 NODE 3 47", "AM-235-N17 contigs AM-235-N17 NODE 10 7", "AH-470-B06 contigs AH-470-B06 NODE 11 7", "AH-470-M19 contigs AH-470-M19 NODE 23 4", "AH-470-N10 contigs AH-470-N10 NODE 1 47", "AH-985-I08 contigs AH-985-I08 NODE 14 11", "AH-985-C14 contigs AH-985-C14 NODE 2 20", "AM-235-C18 contigs AM-235-C18 NODE 4 45", "AH-613-A20 contigs AH-613-A20 NODE 8 12", "AH-982-B09 contigs AH-982-B09 NODE 3 39", "AH-985-N06 contigs AH-985-N06 NODE 7 54", "AM-276-D04 contigs AM-276-D04 NODE 1 24", "AM-276-J16 contigs AM-276-J16 NODE 7 12", "AM-685-M06 contigs SCGC AM-685-M06 contig16 8", "AH-473-N15 contigs AH-473-N15 NODE 8 38", "AH-473-F10 contigs AH-473-F10 NODE 5 13", "AH-982-G18 contigs AH-982-G18 NODE 6 8", "AH-985-G10 contigs AH-985-G10 NODE 9 10", "AH-470-G13 contigs AH-470-G13 NODE 9 6", "AM-264-E22 contigs AM-264-E22 NODE 7 8", "AM-247-L05 contigs AM-247-L05 NODE 22 1", "AM-187-K11 contigs AM-187-K11 NODE 2 14", "AM-264-C11 contigs AM-264-C11 NODE 1 39", "AM-685-I08 contigs SCGC AM-685-I08 contig20 7", "AM-235-N11 contigs AM-235-N11 NODE 31 3", "AM-118-N09 contigs AM-118-N09 NODE 1 7", "AM-208-N16 contigs AM-208-N16 NODE 4 13", "AM-118-L17 contigs AM-118-L17 NODE 26 6", "AM-262-E04 contigs AM-262-E04 NODE 12 4", "AM-334-F10 contigs AM-334-F10 NODE 4 4", "AM-334-K03 contigs AM-334-K03 NODE 11 11", "AH-470-G14 contigs AH-470-G14 NODE 2 14", "AH-470-P20 contigs AH-470-P20 NODE 3 4", "AM-235-C03 contigs AM-235-C03 NODE 1 35", "AH-613-A14 contigs AH-613-A14 NODE 2 29", "AM-216-D21 contigs AM-216-D21 NODE 4 68", "AM-235-P09 contigs AM-235-P09 NODE 11 10", "AM-235-E10 contigs AM-235-E10 NODE 15 8", "AM-235-O04 contigs AM-235-O04 NODE 13 11", "AM-235-M03 contigs AM-235-M03 NODE 9 19", "AM-235-E23 contigs AM-235-E23 NODE 11 10", "AM-229-I11 contigs AM-229-I11 NODE 20 11", "AH-611-I17 contigs AH-611-I17 NODE 27 7", "AH-613-K14 contigs AH-613-K14 NODE 13 13", "AM-247-J19 contigs AM-247-J19 NODE 7 14", "AM-187-I21 contigs AM-187-I21 NODE 16 6", "AM-138-L20 contigs AM-138-L20 NODE 28 3", "AM-286-F19 contigs AM-286-F19 NODE 1 21", "AM-262-D11 contigs AM-262-D11 NODE 1 35", "AM-247-J13 contigs AM-247-J13 NODE 5 9", "AM-225-P05 contigs AM-225-P05 NODE 3 16", "AH-735-P08 contigs AH-735-P08 NODE 30 1", "AH-735-M10 contigs AH-735-M10 NODE 14 6", "AH-573-P17 contigs AH-573-P17 NODE 5 40", "AM-187-A17 contigs AM-187-A17 NODE 48 3", "AM-201-B23 contigs AM-201-B23 NODE 1 4", "AM-229-G13 contigs AM-229-G13 NODE 3 6", "AH-613-L02 contigs AH-613-L02 NODE 15 10", "AH-985-K15 contigs AH-985-K15 NODE 10 23", "AM-206-G06 contigs AM-206-G06 NODE 20 7", "AM-288-C17 contigs AM-288-C17 NODE 5 12", "AM-164-O15 contigs AM-164-O15 NODE 25 4", "AM-262-M09 contigs AM-262-M09 NODE 8 6", "AM-235-O03 contigs AM-235-O03 NODE 1 61", "AM-201-O10 contigs AM-201-O10 NODE 1 47", "AH-305-K19 contigs AH-305-K19 NODE 3 6", "AH-573-A05 contigs AH-573-A05 NODE 5 38", "AM-206-E11 contigs AM-206-E11 NODE 2 25", "AM-264-E15 contigs AM-264-E15 NODE 53 1", "AM-184-P01 contigs AM-184-P01 NODE 4 30", "AM-184-P10 contigs AM-184-P10 NODE 19 6", "AH-562-B18 contigs AH-562-B18 NODE 9 23", "AM-255-F10 contigs AM-255-F10 NODE 46 1", "AM-138-K02 contigs AM-138-K02 NODE 3 6", "AM-334-K17 contigs AM-334-K17 NODE 6 4", "AM-249-J22 contigs AM-249-J22 NODE 1 25", "AH-470-K16 contigs AH-470-K16 NODE 8 6", "AH-985-B06 contigs AH-985-B06 NODE 2 77", "AM-288-K14 contigs AM-288-K14 NODE 2 27", "AM-225-J05 contigs AM-225-J05 NODE 12 6", "AM-247-K04 contigs AM-247-K04 NODE 16 1", "AM-225-D02 contigs AM-225-D02 NODE 16 2", "2L6X 1|Chain A|Green-light absorbing proteorhodopsin|uncultured marine gamma proteobacterium EBAC31A08 (133804) ", "AH-982-K14 contigs AH-982-K14 NODE 2 13", "AH-982-L14 contigs AH-982-L14 NODE 24 7", "AH-985-D10 contigs AH-985-D10 NODE 1 25", "AH-985-I04 contigs AH-985-I04 NODE 3 29", "AM-249-G03 contigs AM-249-G03 NODE 15 11", "AM-332-K07 contigs AM-332-K07 NODE 5 45", "AM-332-N10 contigs AM-332-N10 NODE 1 26", "AH-470-P19 contigs AH-470-P19 NODE 7 12", "AH-991-O04 contigs AH-991-O04 NODE 5 33", "AH-985-B08 contigs AH-985-B08 NODE 6 26", "AH-985-G03 contigs AH-985-G03 NODE 2 140", "AH-985-A03 contigs AH-985-A03 NODE 16 17", "AH-985-G05 contigs AH-985-G05 NODE 5 27", "AM-235-C04 contigs AM-235-C04 NODE 1 95", "AM-235-E08 contigs AM-235-E08 NODE 4 51", "AM-235-I10 contigs AM-235-I10 NODE 12 24", "AM-235-E05 contigs AM-235-E05 NODE 2 5", "AM-235-J17 contigs AM-235-J17 NODE 3 39", "AM-225-M23 contigs AM-225-M23 NODE 1 72", "AM-262-B11 contigs AM-262-B11 NODE 3 46", "AM-247-K13 contigs AM-247-K13 NODE 9 6", "AM-201-N07 contigs AM-201-N07 NODE 2 16", "AM-216-D03 contigs AM-216-D03 NODE 26 13", "AH-302-M20 contigs AH-302-M20 NODE 2 51", "AM-255-G15 contigs AM-255-G15 NODE 22 5", "AH-888-F10 contigs AH-888-F10 NODE 20 5", "AM-208-B20 contigs AM-208-B20 NODE 36 1", "AM-247-L03 contigs AM-247-L03 NODE 2 27", "AH-991-N14 contigs AH-991-N14 NODE 4 25", "AM-334-G02 contigs AM-334-G02 NODE 15 11", "AM-276-J22 contigs AM-276-J22 NODE 7 9", "AH-613-D20 contigs AH-613-D20 NODE 11 9", "AM-235-E02 contigs AM-235-E02 NODE 2 8", "AH-473-K22 contigs AH-473-K22 NODE 11 9", "AH-473-P19 contigs AH-473-P19 NODE 19 6", "AH-470-K14 contigs AH-470-K14 NODE 10 17", "AH-611-L08 contigs AH-611-L08 NODE 22 11", "AM-216-E19 contigs AM-216-E19 NODE 2 58", "AH-611-C09 contigs AH-611-C09 NODE 33 4", "AH-613-E18 contigs AH-613-E18 NODE 21 10", "AH-611-N13 contigs AH-611-N13 NODE 4 83", "AM-216-O15 contigs AM-216-O15 NODE 69 2", "AM-332-K22 contigs AM-332-K22 NODE 84 3", "AM-334-L09 contigs AM-334-L09 NODE 11 13", "AM-390-K04 contigs AM-390-K04 NODE 144 1", "AM-390-N10 contigs AM-390-N10 NODE 37 2", "AM-390-N10 contigs AM-390-N10 NODE 11 22", "AM-685-I13 contigs SCGC AM-685-I13 contig155 7", "AM-388-E15 contigs AM-388-E15 NODE 18 8", "AM-388-E13 contigs AM-388-E13 NODE 96 4", "AM-388-E13 contigs AM-388-E13 NODE 41 12", "AM-689-G20 contigs SCGC AM-689-G20 contig37 7", "AM-685-N21 contigs SCGC AM-685-N21 contig3 18", "AH-470-P09 contigs AH-470-P09 NODE 19 5", "AH-613-J13 contigs AH-613-J13 NODE 27 2", "AH-613-J13 contigs AH-613-J13 NODE 27 8", "AH-982-J08 contigs AH-982-J08 NODE 5 2", "AH-613-M18 contigs AH-613-M18 NODE 35 2", "AM-235-A19 contigs AM-235-A19 NODE 30 2", "AH-613-M18 contigs AH-613-M18 NODE 35 5", "AH-613-O14 contigs AH-613-O14 NODE 1 17", "AM-229-B10 contigs AM-229-B10 NODE 1 16", "AM-229-F05 contigs AM-229-F05 NODE 1 93", "AM-243-K20 contigs AM-243-K20 NODE 12 8", "AM-235-C15 contigs AM-235-C15 NODE 12 27", "AM-388-F07 contigs AM-388-F07 NODE 9 33", "AM-388-I04 contigs AM-388-I04 NODE 5 24", "AH-611-J11 contigs AH-611-J11 NODE 12 6", "AH-611-N17 contigs AH-611-N17 NODE 13 1", "AH-611-N17 contigs AH-611-N17 NODE 26 8", "AM-388-D06 contigs AM-388-D06 NODE 9 3", "AM-390-K03 contigs AM-390-K03 NODE 6 42", "AM-388-N06 contigs AM-388-N06 NODE 5 21", "AM-388-O03 contigs AM-388-O03 NODE 2 60", "AM-388-O19 contigs AM-388-O19 NODE 5 66", "AM-390-G11 contigs AM-390-G11 NODE 3 42", "AM-388-G18 contigs AM-388-G18 NODE 7 28", "AM-388-I02 contigs AM-388-I02 NODE 4 30", "AM-390-I08 contigs AM-390-I08 NODE 17 2", "AM-390-L11 contigs AM-390-L11 NODE 31 7", "AM-201-P14 contigs AM-201-P14 NODE 2 22", "AM-276-F17 contigs AM-276-F17 NODE 3 56", "AM-388-D13 contigs AM-388-D13 NODE 7 8", "AM-235-L17 contigs AM-235-L17 NODE 31 5", "AH-611-J11 contigs AH-611-J11 NODE 3 27", "AH-473-C06 contigs AH-473-C06 NODE 1 137", "AH-982-A16 contigs AH-982-A16 NODE 15 2", "AH-985-F16 contigs AH-985-F16 NODE 3 65", "AM-235-L04 contigs AM-235-L04 NODE 71 3", "AM-388-E22 contigs AM-388-E22 NODE 3 24", "AM-388-M09 contigs AM-388-M09 NODE 4 31", "AM-390-O23 contigs AM-390-O23 NODE 3 93", "AM-388-O19 contigs AM-388-O19 NODE 8 36", "AM-390-A03 contigs AM-390-A03 NODE 19 17", "AM-390-G11 contigs AM-390-G11 NODE 49 5", "AM-390-M10 contigs AM-390-M10 NODE 8 35", "AM-390-G15 contigs AM-390-G15 NODE 14 9", "AM-235-L22 contigs AM-235-L22 NODE 18 9", "AM-332-C18 contigs AM-332-C18 NODE 18 2", "AM-390-L10 contigs AM-390-L10 NODE 2 31", "AM-332-I17 contigs AM-332-I17 NODE 1 24", "AM-158-K03 contigs AM-158-K03 NODE 11 13", "AM-276-B16 contigs AM-276-B16 NODE 99 1", "AM-201-P14 contigs AM-201-P14 NODE 1 74", "AM-276-F17 contigs AM-276-F17 NODE 7 19", "AM-235-E04 contigs AM-235-E04 NODE 5 1", "AM-332-J08 contigs AM-332-J08 NODE 48 6", "AM-332-L05 contigs AM-332-L05 NODE 64 3", "AM-332-J02 contigs AM-332-J02 NODE 17 1", "AM-243-M17 contigs AM-243-M17 NODE 2 29", "AM-332-K04 contigs AM-332-K04 NODE 7 39", "AM-332-D22 contigs AM-332-D22 NODE 6 6", "AM-685-M09 contigs SCGC AM-685-M09 contig9 44", "AM-235-D03 contigs AM-235-D03 NODE 1 1", "AH-470-O06 contigs AH-470-O06 NODE 6 68", "AM-334-A08 contigs AM-334-A08 NODE 34 6", "AM-235-J06 contigs AM-235-J06 NODE 13 6", "AM-332-O16 contigs AM-332-O16 NODE 9 12", ]

#### LIST of rhodopsins belonging to the 2Fe2s clade, based on the rhodopsin phylogeny
# (In itol, got this by clicking at the ancestral node of the 2Fe2S-rhodopsin clade and selecting [copy leaf labels] from the dropdown menu that unfolds when that node gets clicked)
# https://itol.embl.de/tree/12812822615690281732644278#
LIST_2Fe2S_rhodopsins=["AM-332-B10 contigs AM-332-B10 NODE 8 9", "AM-332-A16 contigs AM-332-A16 NODE 16 2", "AM-334-C06 contigs AM-334-C06 NODE 11 9", "AM-332-L17 contigs AM-332-L17 NODE 8 10", "AM-332-N13 contigs AM-332-N13 NODE 3 11", "AM-235-B08 contigs AM-235-B08 NODE 6 15", "AM-332-N20 contigs AM-332-N20 NODE 1 20", "AM-332-B18 contigs AM-332-B18 NODE 12 2", "AM-259-O02 contigs AM-259-O02 NODE 1 31", "AM-130-G06 contigs AM-130-G06 NODE 6 65", "AH-888-L04 contigs AH-888-L04 NODE 15 6", "AH-735-N13 contigs AH-735-N13 NODE 29 5", "AM-247-J07 contigs AM-247-J07 NODE 30 6", "AM-164-N21 contigs AM-164-N21 NODE 43 6", "AM-288-P17 contigs AM-288-P17 NODE 2 39", "AH-985-A17 contigs AH-985-A17 NODE 1 74", "AH-470-P15 contigs AH-470-P15 NODE 3 1", "AM-332-F18 contigs AM-332-F18 NODE 2 10", "AM-332-C06 contigs AM-332-C06 NODE 4 4", "AM-288-B03 contigs AM-288-B03 NODE 7 10", "AM-130-K09 contigs AM-130-K09 NODE 64 1", "AM-332-A22 contigs AM-332-A22 NODE 1 15", "AM-332-N08 contigs AM-332-N08 NODE 21 5", "AM-334-N11 contigs AM-334-N11 NODE 1 11", "AM-334-A01 contigs AM-334-A01 NODE 10 10", "AM-332-M16 contigs AM-332-M16 NODE 6 5", "AM-332-L08 contigs AM-332-L08 NODE 2 7", "AM-332-F21 contigs AM-332-F21 NODE 6 37", "AM-332-D03 contigs AM-332-D03 NODE 11 29", "AM-332-D02 contigs AM-332-D02 NODE 29 1", "AH-611-C05 contigs AH-611-C05 NODE 43 6", "AH-473-C07 contigs AH-473-C07 NODE 5 25", "AH-611-O09 contigs AH-611-O09 NODE 4 21", "AM-235-A09 contigs AM-235-A09 NODE 37 4", "AM-235-M17 contigs AM-235-M17 NODE 15 7", "AH-611-P07 contigs AH-611-P07 NODE 10 4", "AH-611-I23 contigs AH-611-I23 NODE 4 21", "AH-470-A20 contigs AH-470-A20 NODE 34 1", "AH-985-E07 contigs AH-985-E07 NODE 5 33", "AH-611-N23 contigs AH-611-N23 NODE 2 61", "AH-985-E22 contigs AH-985-E22 NODE 5 10", "AM-235-K23 contigs AM-235-K23 NODE 3 24", "AH-991-E06 contigs AH-991-E06 NODE 9 14", "AH-473-J14 contigs AH-473-J14 NODE 2 22", "AH-470-N15 contigs AH-470-N15 NODE 3 14", "AH-470-I05 contigs AH-470-I05 NODE 3 24", "AH-470-F08 contigs AH-470-F08 NODE 4 18", "AH-985-O05 contigs AH-985-O05 NODE 3 98", "AH-985-B18 contigs AH-985-B18 NODE 9 37", "AH-473-O05 contigs AH-473-O05 NODE 1 56", "AH-473-I07 contigs AH-473-I07 NODE 7 3", "AH-473-D09 contigs AH-473-D09 NODE 22 4", "AH-473-F04 contigs AH-473-F04 NODE 4 28", "AM-332-G10 contigs AM-332-G10 NODE 17 14", "AM-332-P04 contigs AM-332-P04 NODE 9 9", "AH-473-J16 contigs AH-473-J16 NODE 17 6", "AH-473-A19 contigs AH-473-A19 NODE 2 13", "AH-470-J19 contigs AH-470-J19 NODE 1 19", "AH-991-L03 contigs AH-991-L03 NODE 9 5", "AH-470-C17 contigs AH-470-C17 NODE 26 12", "AH-468-K22 contigs AH-468-K22 NODE 12 6", "AH-982-I21 contigs AH-982-I21 NODE 1 111", "AM-235-D23 contigs AM-235-D23 NODE 45 1", "AH-473-N19 contigs AH-473-N19 NODE 7 5", "AH-473-F18 contigs AH-473-F18 NODE 5 35", "AH-470-O19 contigs AH-470-O19 NODE 17 3", "AH-982-C03 contigs AH-982-C03 NODE 1 59", "AH-982-G22 contigs AH-982-G22 NODE 2 170", "AH-470-P11 contigs AH-470-P11 NODE 6 45", "AH-985-D18 contigs AH-985-D18 NODE 3 60", "AM-262-C09 contigs AM-262-C09 NODE 10 9", "AM-332-I16 contigs AM-332-I16 NODE 4 12", "AM-332-D08 contigs AM-332-D08 NODE 14 9", "AM-332-K06 contigs AM-332-K06 NODE 3 14", "AM-332-G22 contigs AM-332-G22 NODE 2 60", "AM-235-G21 contigs AM-235-G21 NODE 12 20", "AH-991-K03 contigs AH-991-K03 NODE 2 46", "AM-201-N19 contigs AM-201-N19 NODE 5 6", "AM-247-L05 contigs AM-247-L05 NODE 22 1", "AH-470-M19 contigs AH-470-M19 NODE 23 4", "AM-235-C18 contigs AM-235-C18 NODE 4 45", "AH-470-N10 contigs AH-470-N10 NODE 1 47", "AH-985-I08 contigs AH-985-I08 NODE 14 11", "AH-985-C14 contigs AH-985-C14 NODE 2 20", "AH-470-B06 contigs AH-470-B06 NODE 11 7", "AH-470-G13 contigs AH-470-G13 NODE 9 6", "AH-473-N15 contigs AH-473-N15 NODE 8 38", "AH-985-N06 contigs AH-985-N06 NODE 7 54", "AM-264-E22 contigs AM-264-E22 NODE 7 8", "AM-685-M06 contigs SCGC AM-685-M06 contig16 8", "AM-276-J16 contigs AM-276-J16 NODE 7 12", "AM-276-D04 contigs AM-276-D04 NODE 1 24", "AH-982-B09 contigs AH-982-B09 NODE 3 39", "AH-613-A20 contigs AH-613-A20 NODE 8 12", "AH-985-G10 contigs AH-985-G10 NODE 9 10", "AH-982-G18 contigs AH-982-G18 NODE 6 8", "AH-473-F10 contigs AH-473-F10 NODE 5 13", "AM-235-N17 contigs AM-235-N17 NODE 10 7", "AH-473-M06 contigs AH-473-M06 NODE 3 47", "AH-470-K09 contigs AH-470-K09 NODE 24 5", "AH-985-C23 contigs AH-985-C23 NODE 7 17", "AH-470-B04 contigs AH-470-B04 NODE 24 3", "AH-613-F09 contigs AH-613-F09 NODE 36 3", "AM-332-C20 contigs AM-332-C20 NODE 3 54", "AM-187-P09 contigs AM-187-P09 NODE 2 10", "AM-247-E21 contigs AM-247-E21 NODE 2 64", "AH-613-E20 contigs AH-613-E20 NODE 13 9", "AM-201-A14 contigs AM-201-A14 NODE 20 4", "AH-613-E11 contigs AH-613-E11 NODE 44 3", "AM-685-F13 contigs SCGC AM-685-F13 contig16 8", "AH-735-I22 contigs AH-735-I22 NODE 12 5", "AM-334-O04 contigs AM-334-O04 NODE 7 15", "AM-334-M02 contigs AM-334-M02 NODE 10 8", "AM-164-D11 contigs AM-164-D11 NODE 5 20", "AM-225-M07 contigs AM-225-M07 NODE 45 1", "AM-225-A20 contigs AM-225-A20 NODE 2 8", "AM-187-J20 contigs AM-187-J20 NODE 47 1", "AM-187-G11 contigs AM-187-G11 NODE 6 31", "AG-538-J23 contigs AG-538-J23 NODE 9 13", "AM-249-J22 contigs AM-249-J22 NODE 1 25", "AM-262-D11 contigs AM-262-D11 NODE 1 35", "AM-229-I11 contigs AM-229-I11 NODE 20 11", "AM-286-F19 contigs AM-286-F19 NODE 1 21", "AM-138-L20 contigs AM-138-L20 NODE 28 3", "AM-247-J19 contigs AM-247-J19 NODE 7 14", "AH-613-K14 contigs AH-613-K14 NODE 13 13", "AM-187-I21 contigs AM-187-I21 NODE 16 6", "AH-611-I17 contigs AH-611-I17 NODE 27 7", "AM-138-K04 contigs AM-138-K04 NODE 10 19", "AM-130-I22 contigs AM-130-I22 NODE 14 8", "AM-124-K09 contigs AM-124-K09 NODE 2 6", "AM-201-F22 contigs AM-201-F22 NODE 25 6", "AH-562-O14 contigs AH-562-O14 NODE 25 4", "AH-562-C07 contigs AH-562-C07 NODE 38 7", "AM-216-O05 contigs AM-216-O05 NODE 7 4", "AH-562-K21 contigs AH-562-K21 NODE 7 20", "AM-334-O03 contigs AM-334-O03 NODE 1 55", "AH-888-A19 contigs AH-888-A19 NODE 3 15", "AM-288-O07 contigs AM-288-O07 NODE 2 37", "AM-121-N18 contigs AM-121-N18 NODE 5 23", "AH-888-G20 contigs AH-888-G20 NODE 29 2", "AM-138-G09 contigs AM-138-G09 NODE 34 7", "AM-286-F05 contigs AM-286-F05 NODE 1 41", "AH-562-E04 contigs AH-562-E04 NODE 23 5", "AH-735-G19 contigs AH-735-G19 NODE 1 1", "AM-247-J13 contigs AM-247-J13 NODE 5 9", "AM-138-J06 contigs AM-138-J06 NODE 3 17", "AG-538-M09 contigs AG-538-M09 NODE 28 3", "AM-118-E14 contigs AM-118-E14 NODE 6 12", "AM-216-G06 contigs AM-216-G06 NODE 2 33", "AM-164-M18 contigs AM-164-M18 NODE 6 28", "AM-286-G07 contigs AM-286-G07 NODE 1 25", "AH-613-E07 contigs AH-613-E07 NODE 18 12", "AM-216-E08 contigs AM-216-E08 NODE 31 6", "AH-735-K15 contigs AH-735-K15 NODE 2 5", "AM-334-B22 contigs AM-334-B22 NODE 3 22", "AH-888-D11 contigs AH-888-D11 NODE 14 1", "AM-247-K15 contigs AM-247-K15 NODE 5 6", "AH-573-J17 contigs AH-573-J17 NODE 13 27", "AM-243-B08 contigs AM-243-B08 NODE 21 3", "AH-735-A10 contigs AH-735-A10 NODE 2 9", "AM-247-P05 contigs AM-247-P05 NODE 1 23", "AM-243-A06 contigs AM-243-A06 NODE 1 62", "AM-187-B23 contigs AM-187-B23 NODE 23 8", "AM-235-J04 contigs AM-235-J04 NODE 59 1", "AH-613-J18 contigs AH-613-J18 NODE 9 7", "AM-685-E15 contigs SCGC AM-685-E15 contig5 25", "AM-235-C09 contigs AM-235-C09 NODE 40 1", "AH-611-P06 contigs AH-611-P06 NODE 3 17", "AH-735-L16 contigs AH-735-L16 NODE 2 28", "AH-613-C13 contigs AH-613-C13 NODE 7 13", "AM-685-F04 contigs SCGC AM-685-F04 contig7 14", "AM-235-O15 contigs AM-235-O15 NODE 1 24", "AH-735-O16 contigs AH-735-O16 NODE 1 23", "AM-243-F23 contigs AM-243-F23 NODE 3 5", "AM-286-N18 contigs AM-286-N18 NODE 4 34", "AM-247-K09 contigs AM-247-K09 NODE 18 15", "AH-888-C20 contigs AH-888-C20 NODE 1 33", "AH-991-J22 contigs AH-991-J22 NODE 3 13", "AM-235-P18 contigs AM-235-P18 NODE 1 65", "AM-235-F14 contigs AM-235-F14 NODE 2 22", "AM-187-M17 contigs AM-187-M17 NODE 7 4", "AM-187-K08 contigs AM-187-K08 NODE 6 7", "AH-888-O06 contigs AH-888-O06 NODE 1 82", "AH-735-A03 contigs AH-735-A03 NODE 13 1", "AM-689-N03 contigs SCGC AM-689-N03 contig1 25", "AH-735-L20 contigs AH-735-L20 NODE 12 19", "AM-187-F11 contigs AM-187-F11 NODE 3 15", "AM-184-O13 contigs AM-184-O13 NODE 1 26", "AM-118-B02 contigs AM-118-B02 NODE 29 7", "AH-735-G21 contigs AH-735-G21 NODE 2 64", "AM-229-O19 contigs AM-229-O19 NODE 1 66", "AM-243-C08 contigs AM-243-C08 NODE 4 23", "AH-613-N10 contigs AH-613-N10 NODE 74 2", "AM-201-K09 contigs AM-201-K09 NODE 20 2", "AM-286-K06 contigs AM-286-K06 NODE 1 31", "AM-286-F07 contigs AM-286-F07 NODE 1 59", "AM-235-D15 contigs AM-235-D15 NODE 6 13", "AM-288-J17 contigs AM-288-J17 NODE 1 17", "AM-249-B18 contigs AM-249-B18 NODE 8 20", "AH-888-L22 contigs AH-888-L22 NODE 6 17", "AH-735-M23 contigs AH-735-M23 NODE 2 22", "AM-243-E13 contigs AM-243-E13 NODE 13 4", "AH-985-I18 contigs AH-985-I18 NODE 3 27", "AM-235-L08 contigs AM-235-L08 NODE 7 29", "AM-235-D14 contigs AM-235-D14 NODE 4 11", "AM-334-B19 contigs AM-334-B19 NODE 15 1", "AM-284-A18 contigs AM-284-A18 NODE 43 6", "AH-735-D20 contigs AH-735-D20 NODE 20 7", "AM-130-E23 contigs AM-130-E23 NODE 6 20", "AM-243-F04 contigs AM-243-F04 NODE 15 6", "AM-210-A17 contigs AM-210-A17 NODE 12 15", "AM-334-K17 contigs AM-334-K17 NODE 6 4", "AM-138-K02 contigs AM-138-K02 NODE 3 6", "AM-255-F10 contigs AM-255-F10 NODE 46 1", "AM-201-O10 contigs AM-201-O10 NODE 1 47", "AH-562-B18 contigs AH-562-B18 NODE 9 23", "AM-206-E11 contigs AM-206-E11 NODE 2 25", "AH-573-A05 contigs AH-573-A05 NODE 5 38", "AH-305-K19 contigs AH-305-K19 NODE 3 6", "AM-264-E15 contigs AM-264-E15 NODE 53 1", "AM-184-P10 contigs AM-184-P10 NODE 19 6", "AM-184-P01 contigs AM-184-P01 NODE 4 30", "AM-288-C17 contigs AM-288-C17 NODE 5 12", "AM-206-G06 contigs AM-206-G06 NODE 20 7", "AM-235-O03 contigs AM-235-O03 NODE 1 61", "AM-262-M09 contigs AM-262-M09 NODE 8 6", "AM-164-O15 contigs AM-164-O15 NODE 25 4", "AM-201-B23 contigs AM-201-B23 NODE 1 4", "AH-613-L02 contigs AH-613-L02 NODE 15 10", "AM-229-G13 contigs AM-229-G13 NODE 3 6", "AH-573-P17 contigs AH-573-P17 NODE 5 40", "AH-985-K15 contigs AH-985-K15 NODE 10 23", "AM-187-A17 contigs AM-187-A17 NODE 48 3", "AH-735-P08 contigs AH-735-P08 NODE 30 1", "AH-735-M10 contigs AH-735-M10 NODE 14 6", "AM-235-N11 contigs AM-235-N11 NODE 31 3", "AH-735-E21 contigs AH-735-E21 NODE 15 1", "AM-118-J17 contigs AM-118-J17 NODE 11 23", "AM-334-O18 contigs AM-334-O18 NODE 36 6", "AG-538-B02 contigs AG-538-B02 NODE 14 4", "AH-735-M08 contigs AH-735-M08 NODE 19 10", "AH-735-K11 contigs AH-735-K11 NODE 4 13", "AM-118-L17 contigs AM-118-L17 NODE 26 6", "AM-208-N16 contigs AM-208-N16 NODE 4 13", "AM-262-E04 contigs AM-262-E04 NODE 12 4", "AM-118-N09 contigs AM-118-N09 NODE 1 7", "AM-334-K03 contigs AM-334-K03 NODE 11 11", "AM-334-F10 contigs AM-334-F10 NODE 4 4", "AH-302-P11 contigs AH-302-P11 NODE 6 24", "AH-305-P08 contigs AH-305-P08 NODE 2 27", "AM-208-L13 contigs AM-208-L13 NODE 13 10", "AM-276-M20 contigs AM-276-M20 NODE 23 17", "AM-280-J04 contigs AM-280-J04 NODE 36 1", "AM-259-G09 contigs AM-259-G09 NODE 1 27", "AM-276-N02 contigs AM-276-N02 NODE 3 36", "AM-249-I23 contigs AM-249-I23 NODE 4 13", "AM-127-L08 contigs AM-127-L08 NODE 54 6", "AM-334-E18 contigs AM-334-E18 NODE 1 26", "AM-235-I03 contigs AM-235-I03 NODE 31 2", "AM-243-A07 contigs AM-243-A07 NODE 5 36", "AM-164-L02 contigs AM-164-L02 NODE 1 29", "AH-611-D02 contigs AH-611-D02 NODE 22 5", "AM-685-G04 contigs SCGC AM-685-G04 contig7 1", "AM-262-N05 contigs AM-262-N05 NODE 6 1", "AH-735-P04 contigs AH-735-P04 NODE 60 1", "AM-235-J07 contigs AM-235-J07 NODE 9 13", "AH-888-O18 contigs AH-888-O18 NODE 13 3", "AH-735-A01 contigs AH-735-A01 NODE 12 8", "AH-611-L14 contigs AH-611-L14 NODE 5 3", "AM-216-C16 contigs AM-216-C16 NODE 7 20", "AM-187-E15 contigs AM-187-E15 NODE 11 21", "AH-473-B13 contigs AH-473-B13 NODE 22 8", "AH-611-F04 contigs AH-611-F04 NODE 13 7", "AM-118-A20 contigs AM-118-A20 NODE 46 1", "AM-201-G23 contigs AM-201-G23 NODE 4 9", "AH-991-F05 contigs AH-991-F05 NODE 29 4", "AH-735-M18 contigs AH-735-M18 NODE 1 10", "AH-573-N23 contigs AH-573-N23 NODE 12 8", "AM-164-B09 contigs AM-164-B09 NODE 24 6", "AM-264-L18 contigs AM-264-L18 NODE 9 7", "AM-255-P07 contigs AM-255-P07 NODE 53 5", "AM-280-E05 contigs AM-280-E05 NODE 9 7", "AM-201-G09 contigs AM-201-G09 NODE 8 11", "AH-302-C16 contigs AH-302-C16 NODE 2 56", "AM-158-K17 contigs AM-158-K17 NODE 3 8", "AH-988-D21 contigs AH-988-D21 NODE 11 8", "AM-685-I08 contigs SCGC AM-685-I08 contig20 7", "AM-264-C11 contigs AM-264-C11 NODE 1 39", "AM-187-K11 contigs AM-187-K11 NODE 2 14", "AH-991-D11 contigs AH-991-D11 NODE 21 6", "AH-613-I09 contigs AH-613-I09 NODE 4 17", "AM-286-I03 contigs AM-286-I03 NODE 12 1", "AM-216-K08 contigs AM-216-K08 NODE 54 1", "AH-735-E08 contigs AH-735-E08 NODE 3 9", "AM-201-O16 contigs AM-201-O16 NODE 13 10", "AM-288-J07 contigs AM-288-J07 NODE 3 47", "AH-613-I15 contigs AH-613-I15 NODE 8 11", "AM-286-B19 contigs AM-286-B19 NODE 9 12", "AH-735-G22 contigs AH-735-G22 NODE 9 10", "AM-138-J05 contigs AM-138-J05 NODE 14 13", "AM-138-J05 contigs AM-138-J05 NODE 14 7", "AM-184-C05 contigs AM-184-C05 NODE 7 8", "AH-735-K19 contigs AH-735-K19 NODE 5 13", "AM-208-K11 contigs AM-208-K11 NODE 27 6", "AM-247-B14 contigs AM-247-B14 NODE 1 43", "AH-611-J16 contigs AH-611-J16 NODE 2 19", "AM-334-G02 contigs AM-334-G02 NODE 15 11", "AM-208-B20 contigs AM-208-B20 NODE 36 1", "AM-247-L03 contigs AM-247-L03 NODE 2 27", "AM-262-B11 contigs AM-262-B11 NODE 3 46", "AM-247-K13 contigs AM-247-K13 NODE 9 6", "AM-225-M23 contigs AM-225-M23 NODE 1 72", "AM-225-J05 contigs AM-225-J05 NODE 12 6", "AM-288-K14 contigs AM-288-K14 NODE 2 27", "AM-247-K04 contigs AM-247-K04 NODE 16 1", "AM-225-P05 contigs AM-225-P05 NODE 3 16", "AH-985-B06 contigs AH-985-B06 NODE 2 77", "AH-470-K16 contigs AH-470-K16 NODE 8 6", "AH-470-P20 contigs AH-470-P20 NODE 3 4", "AM-235-C03 contigs AM-235-C03 NODE 1 35", "AH-470-G14 contigs AH-470-G14 NODE 2 14", "AM-235-O04 contigs AM-235-O04 NODE 13 11", "AH-613-A14 contigs AH-613-A14 NODE 2 29", "AM-235-M03 contigs AM-235-M03 NODE 9 19", "AM-216-D21 contigs AM-216-D21 NODE 4 68", "AM-235-E10 contigs AM-235-E10 NODE 15 8", "AM-235-E23 contigs AM-235-E23 NODE 11 10", "AM-235-P09 contigs AM-235-P09 NODE 11 10", "AM-138-C10 contigs AM-138-C10 NODE 30 4", "AM-286-O15 contigs AM-286-O15 NODE 9 11", "AH-573-A18 contigs AH-573-A18 NODE 23 7", "AH-613-J09 contigs AH-613-J09 NODE 12 21", "AH-888-G18 contigs AH-888-G18 NODE 18 8", "AH-735-N06 contigs AH-735-N06 NODE 20 7", "AM-332-I06 contigs AM-332-I06 NODE 5 8", "AM-255-M18 contigs AM-255-M18 NODE 31 6", "AM-201-C21 contigs AM-201-C21 NODE 36 6", "AM-118-M18 contigs AM-118-M18 NODE 11 14", "AM-255-N03 contigs AM-255-N03 NODE 25 9", "AH-991-G23 contigs AH-991-G23 NODE 11 2", "AM-249-I18 contigs AM-249-I18 NODE 25 8", "AM-235-K05 contigs AM-235-K05 NODE 26 4", "AM-286-I22 contigs AM-286-I22 NODE 19 7", "AH-735-G09 contigs AH-735-G09 NODE 22 8", "AH-888-M17 contigs AH-888-M17 NODE 10 4", "AM-235-I09 contigs AM-235-I09 NODE 20 3", "AH-573-F17 contigs AH-573-F17 NODE 1 133", "AM-249-G22 contigs AM-249-G22 NODE 9 1", "AM-247-A03 contigs AM-247-A03 NODE 6 10", "AM-121-O09 contigs AM-121-O09 NODE 22 3", "AM-164-A01 contigs AM-164-A01 NODE 7 4", "AH-573-C21 contigs AH-573-C21 NODE 17 3", "AH-735-J03 contigs AH-735-J03 NODE 15 22", "AM-187-K07 contigs AM-187-K07 NODE 20 8", "AH-985-K08 contigs AH-985-K08 NODE 4 21", "AM-118-M15 contigs AM-118-M15 NODE 7 18", "AM-208-D10 contigs AM-208-D10 NODE 22 11", "AM-288-P10 contigs AM-288-P10 NODE 5 23", "AM-206-N08 contigs AM-206-N08 NODE 12 23", "AM-201-N07 contigs AM-201-N07 NODE 2 16", "AM-225-D02 contigs AM-225-D02 NODE 16 2", "AH-473-K22 contigs AH-473-K22 NODE 11 9", "AH-888-F10 contigs AH-888-F10 NODE 20 5", "AM-255-G15 contigs AM-255-G15 NODE 22 5", "AH-302-M20 contigs AH-302-M20 NODE 2 51", "AM-184-I06 contigs AM-184-I06 NODE 7 1", "AM-161-C03 contigs AM-161-C03 NODE 5 16", "AM-208-C02 contigs AM-208-C02 NODE 49 1", "AH-305-K03 contigs AH-305-K03 NODE 1 39", "AM-208-K04 contigs AM-208-K04 NODE 11 1", "AM-262-I09 contigs AM-262-I09 NODE 1 1", "AH-562-L13 contigs AH-562-L13 NODE 4 53", "AM-216-J22 contigs AM-216-J22 NODE 18 5", "AM-225-L16 contigs AM-225-L16 NODE 28 3", "AM-216-C21 contigs AM-216-C21 NODE 24 5", "AM-210-J07 contigs AM-210-J07 NODE 1 73", "AM-264-E23 contigs AM-264-E23 NODE 9 12", "AM-334-F08 contigs AM-334-F08 NODE 16 12", "AM-118-K20 contigs AM-118-K20 NODE 12 12", "AM-118-P17 contigs AM-118-P17 NODE 3 7", "AM-334-D04 contigs AM-334-D04 NODE 17 6", "AM-187-J02 contigs AM-187-J02 NODE 11 11", "AM-332-C08 contigs AM-332-C08 NODE 8 13", "AM-184-M06 contigs AM-184-M06 NODE 3 16", "AM-280-C14 contigs AM-280-C14 NODE 12 13", "AM-249-M09 contigs AM-249-M09 NODE 76 3", "AG-538-D17 contigs AG-538-D17 NODE 27 3", "AM-288-B14 contigs AM-288-B14 NODE 1 73", "AM-332-G04 contigs AM-332-G04 NODE 9 17", "AM-262-E15 contigs AM-262-E15 NODE 12 11", "AM-118-G13 contigs AM-118-G13 NODE 18 8", "AH-573-M15 contigs AH-573-M15 NODE 20 5", "AM-249-B14 contigs AM-249-B14 NODE 11 9", "AM-216-K15 contigs AM-216-K15 NODE 2 34", "AM-121-E03 contigs AM-121-E03 NODE 32 7", "AM-334-G07 contigs AM-334-G07 NODE 8 4", "AM-216-B07 contigs AM-216-B07 NODE 17 7", "AM-249-K22 contigs AM-249-K22 NODE 1 30", "AH-611-A20 contigs AH-611-A20 NODE 13 33", "AM-187-C18 contigs AM-187-C18 NODE 24 8", "AH-573-P04 contigs AH-573-P04 NODE 2 36", "AH-988-C21 contigs AH-988-C21 NODE 8 18", "AM-243-C16 contigs AM-243-C16 NODE 23 6", "AM-208-K18 contigs AM-208-K18 NODE 9 14", "AM-255-M09 contigs AM-255-M09 NODE 6 19", "AM-255-C18 contigs AM-255-C18 NODE 18 7", "AH-573-A09 contigs AH-573-A09 NODE 10 29", "AM-184-P16 contigs AM-184-P16 NODE 22 5", "AM-264-F14 contigs AM-264-F14 NODE 42 3", "AH-573-O15 contigs AH-573-O15 NODE 7 18", "AM-184-L15 contigs AM-184-L15 NODE 1 56", "AM-216-D10 contigs AM-216-D10 NODE 3 6", "AM-124-F21 contigs AM-124-F21 NODE 37 3", "AH-988-B10 contigs AH-988-B10 NODE 8 10", "AH-735-B03 contigs AH-735-B03 NODE 1 75", "AM-121-P05 contigs AM-121-P05 NODE 2 82", "AM-118-K21 contigs AM-118-K21 NODE 29 2", "AG-538-G09 contigs AG-538-G09 NODE 17 4", "AH-307-I04 contigs AH-307-I04 NODE 6 11", "AH-305-M13 contigs AH-305-M13 NODE 9 6", "AH-302-L09 contigs AH-302-L09 NODE 11 3", "AM-255-O11 contigs AM-255-O11 NODE 2 57", "AM-118-P09 contigs AM-118-P09 NODE 6 25", "AM-334-O09 contigs AM-334-O09 NODE 4 24", "AH-305-K23 contigs AH-305-K23 NODE 9 11", "AM-210-D16 contigs AM-210-D16 NODE 2 12", "AH-307-G21 contigs AH-307-G21 NODE 16 15", "AM-135-L13 contigs AM-135-L13 NODE 3 34", "AM-276-A04 contigs AM-276-A04 NODE 3 8", "AM-184-E04 contigs AM-184-E04 NODE 2 34", "AM-280-L17 contigs AM-280-L17 NODE 1 29", "AM-124-C13 contigs AM-124-C13 NODE 1 53", "AM-158-K11 contigs AM-158-K11 NODE 35 6", "AM-243-K07 contigs AM-243-K07 NODE 45 6", "AM-161-A19 contigs AM-161-A19 NODE 39 1", "AM-127-K13 contigs AM-127-K13 NODE 40 6", "AM-276-F23 contigs AM-276-F23 NODE 5 6", "AM-161-G23 contigs AM-161-G23 NODE 14 19", "AM-124-E05 contigs AM-124-E05 NODE 5 31", "AM-243-J14 contigs AM-243-J14 NODE 4 6", "AM-229-K19 contigs AM-229-K19 NODE 37 2", "AM-276-A01 contigs AM-276-A01 NODE 21 6", "AM-124-I15 contigs AM-124-I15 NODE 53 2", "AH-309-A13 contigs AH-309-A13 NODE 23 14", "AM-284-B07 contigs AM-284-B07 NODE 1 42", "AM-249-N20 contigs AM-249-N20 NODE 7 21", "AH-573-D18 contigs AH-573-D18 NODE 1 7", "AM-187-O05 contigs AM-187-O05 NODE 6 18", "AM-135-M08 contigs AM-135-M08 NODE 13 12", "AH-305-A06 contigs AH-305-A06 NODE 1 19", "AM-280-G11 contigs AM-280-G11 NODE 31 6", "AM-124-G09 contigs AM-124-G09 NODE 36 11", "AM-288-F06 contigs AM-288-F06 NODE 16 8", "AM-249-M11 contigs AM-249-M11 NODE 6 11", "AM-208-C11 contigs AM-208-C11 NODE 8 13", "AM-121-N16 contigs AM-121-N16 NODE 23 6", "AM-121-I21 contigs AM-121-I21 NODE 2 56", "AM-229-F19 contigs AM-229-F19 NODE 1 60", "AM-118-K03 contigs AM-118-K03 NODE 4 20", "AM-280-K21 contigs AM-280-K21 NODE 13 19"]
LIST_FA_rhodopsins=["AH-573-G13 contigs AH-573-G13 NODE 12 26", "AH-991-I11 contigs AH-991-I11 NODE 38 3", "AH-991-E11 contigs AH-991-E11 NODE 9 24", "AH-991-L21 contigs AH-991-L21 NODE 20 2", "AH-985-F10 contigs AH-985-F10 NODE 8 7", "AH-470-F21 contigs AH-470-F21 NODE 38 1", "AH-470-G08 contigs AH-470-G08 NODE 2 79", "AM-235-D09 contigs AM-235-D09 NODE 18 10", "AM-235-A14 contigs AM-235-A14 NODE 4 21", "AH-982-N13 contigs AH-982-N13 NODE 9 9", "AH-473-I02 contigs AH-473-I02 NODE 9 35", "AH-470-I16 contigs AH-470-I16 NODE 34 3", "AH-613-O11 contigs AH-613-O11 NODE 50 5", "AH-991-K18 contigs AH-991-K18 NODE 8 16", "AM-206-C06 contigs AM-206-C06 NODE 3 14", "AH-735-M06 contigs AH-735-M06 NODE 20 7", "AM-235-I17 contigs AM-235-I17 NODE 7 9", "AM-118-J06 contigs AM-118-J06 NODE 51 2", "AH-982-L21 contigs AH-982-L21 NODE 3 51", "AH-982-D18 contigs AH-982-D18 NODE 53 6", "AM-334-E17 contigs AM-334-E17 NODE 4 29", "AM-286-C06 contigs AM-286-C06 NODE 6 20", "AH-613-A18 contigs AH-613-A18 NODE 5 99", "AM-164-F02 contigs AM-164-F02 NODE 10 18", "AM-235-A11 contigs AM-235-A11 NODE 9 15", "AH-982-I17 contigs AH-982-I17 NODE 4 11", "AH-562-N19 contigs AH-562-N19 NODE 17 8", "AM-235-F23 contigs AM-235-F23 NODE 34 6", "AH-562-K09 contigs AH-562-K09 NODE 14 13", "AM-164-J16 contigs AM-164-J16 NODE 4 31", "AH-735-C13 contigs AH-735-C13 NODE 10 35", "AM-235-A22 contigs AM-235-A22 NODE 18 15", "AM-262-C18 contigs AM-262-C18 NODE 50 2", "AM-201-M09 contigs AM-201-M09 NODE 21 7", "AH-562-I11 contigs AH-562-I11 NODE 16 1", "AG-538-O20 contigs AG-538-O20 NODE 6 2", "AM-334-G14 contigs AM-334-G14 NODE 8 2", "AM-334-F22 contigs AM-334-F22 NODE 5 8", "AM-334-B04 contigs AM-334-B04 NODE 37 2", "AM-332-L19 contigs AM-332-L19 NODE 34 2", "AM-332-A08 contigs AM-332-A08 NODE 3 37", "AH-735-P07 contigs AH-735-P07 NODE 19 17", "AG-538-D18 contigs AG-538-D18 NODE 2 12", "AM-332-L21 contigs AM-332-L21 NODE 1 63", "AM-332-J17 contigs AM-332-J17 NODE 6 7", "AM-332-D18 contigs AM-332-D18 NODE 11 9", "AM-332-F16 contigs AM-332-F16 NODE 2 12", "AM-332-E20 contigs AM-332-E20 NODE 4 49", "AH-735-B20 contigs AH-735-B20 NODE 9 22", "AM-216-E15 contigs AM-216-E15 NODE 13 9", "AM-201-C09 contigs AM-201-C09 NODE 10 10", "AM-164-M16 contigs AM-164-M16 NODE 4 18", "AM-216-F09 contigs AM-216-F09 NODE 7 10", "AM-201-L09 contigs AM-201-L09 NODE 6 10", "AG-538-I10 contigs AG-538-I10 NODE 4 34", "AM-216-L09 contigs AM-216-L09 NODE 4 17", "AM-201-M13 contigs AM-201-M13 NODE 25 2", "AG-538-C05 contigs AG-538-C05 NODE 1 19", "AM-286-N13 contigs AM-286-N13 NODE 33 2", "AH-562-K05 contigs AH-562-K05 NODE 14 7", "AH-562-F19 contigs AH-562-F19 NODE 2 16", "AM-164-P10 contigs AM-164-P10 NODE 8 4", "AM-201-I11 contigs AM-201-I11 NODE 13 3", "AH-735-F18 contigs AH-735-F18 NODE 51 3", "AH-735-D04 contigs AH-735-D04 NODE 1 14", "AH-982-I14 contigs AH-982-I14 NODE 57 1", "AM-184-K21 contigs AM-184-K21 NODE 5 21", "AM-121-L17 contigs AM-121-L17 NODE 12 9", "AM-257-M21 contigs AM-257-M21 NODE 13 9", "AM-257-D13 contigs AM-257-D13 NODE 4 29", "AM-121-O06 contigs AM-121-O06 NODE 8 12", "AM-121-F11 contigs AM-121-F11 NODE 4 13", "AH-562-B19 contigs AH-562-B19 NODE 42 4", "AM-332-P10 contigs AM-332-P10 NODE 15 1", "AH-735-J15 contigs AH-735-J15 NODE 1 96", "AM-118-B16 contigs AM-118-B16 NODE 2 8", "AG-538-A02 contigs AG-538-A02 NODE 4 9", "AM-225-N23 contigs AM-225-N23 NODE 6 19", "AH-735-E13 contigs AH-735-E13 NODE 7 49", "AM-225-L22 contigs AM-225-L22 NODE 36 2", "AM-225-G23 contigs AM-225-G23 NODE 2 9", "AM-216-D06 contigs AM-216-D06 NODE 13 3", "AM-118-A22 contigs AM-118-A22 NODE 47 2", "AM-208-B09 contigs AM-208-B09 NODE 22 4", "AM-184-G17 contigs AM-184-G17 NODE 18 19", "AH-573-O17 contigs AH-573-O17 NODE 46 2", "AM-249-A21 contigs AM-249-A21 NODE 15 3", "AM-184-A19 contigs AM-184-A19 NODE 7 7", "AH-991-I16 contigs AH-991-I16 NODE 20 5", "AM-264-M02 contigs AM-264-M02 NODE 3 21", "AM-187-I16 contigs AM-187-I16 NODE 12 10", "AM-138-I02 contigs AM-138-I02 NODE 13 16", "AH-735-J05 contigs AH-735-J05 NODE 18 9", "AH-562-A11 contigs AH-562-A11 NODE 8 1", "AH-888-D21 contigs AH-888-D21 NODE 10 14", "AM-208-I08 contigs AM-208-I08 NODE 10 11", "AM-206-A16 contigs AM-206-A16 NODE 4 11", "AM-216-L20 contigs AM-216-L20 NODE 6 14", "AM-138-B23 contigs AM-138-B23 NODE 11 8", "AH-735-G06 contigs AH-735-G06 NODE 19 10", "AH-562-K18 contigs AH-562-K18 NODE 15 3", "AH-613-K21 contigs AH-613-K21 NODE 6 13", "AM-332-I03 contigs AM-332-I03 NODE 7 19", "AM-332-G21 contigs AM-332-G21 NODE 9 15", "AM-332-G10 contigs AM-332-G10 NODE 33 6", "AH-985-K18 contigs AH-985-K18 NODE 14 10", "AH-473-J23 contigs AH-473-J23 NODE 6 42", "AH-985-C14 contigs AH-985-C14 NODE 16 12", "AH-470-M19 contigs AH-470-M19 NODE 19 6", "AH-470-B06 contigs AH-470-B06 NODE 16 10", "AH-473-K11 contigs AH-473-K11 NODE 1 62", "AH-985-I08 contigs AH-985-I08 NODE 12 10", "AH-473-K22 contigs AH-473-K22 NODE 12 7", "AH-473-O08 contigs AH-473-O08 NODE 2 15", "AH-985-C03 contigs AH-985-C03 NODE 5 25", "AH-985-A17 contigs AH-985-A17 NODE 39 2", "AM-235-C18 contigs AM-235-C18 NODE 9 20", "AM-235-J03 contigs AM-235-J03 NODE 7 22", "AH-982-C03 contigs AH-982-C03 NODE 1 217", "AH-735-F02 contigs AH-735-F02 NODE 35 1", "AH-562-L13 contigs AH-562-L13 NODE 1 33", "AM-201-D19 contigs AM-201-D19 NODE 8 1", "AM-334-G07 contigs AM-334-G07 NODE 37 1", "AM-208-C02 contigs AM-208-C02 NODE 16 20", "AM-288-P10 contigs AM-288-P10 NODE 3 1", "AM-288-A06 contigs AM-288-A06 NODE 35 1", "AM-255-O15 contigs AM-255-O15 NODE 16 1", "AM-255-I17 contigs AM-255-I17 NODE 5 16", "AM-187-J02 contigs AM-187-J02 NODE 2 45", "AM-121-K17 contigs AM-121-K17 NODE 4 26", "AM-121-E03 contigs AM-121-E03 NODE 23 10", "AM-121-E03 contigs AM-121-E03 NODE 23 1", "AM-121-A19 contigs AM-121-A19 NODE 20 1", "AM-118-K20 contigs AM-118-K20 NODE 5 8", "AH-573-P04 contigs AH-573-P04 NODE 4 30", "AH-307-L06 contigs AH-307-L06 NODE 2 56", "AM-334-G06 contigs AM-334-G06 NODE 4 16", "AM-216-C21 contigs AM-216-C21 NODE 10 1", "AM-259-M11 contigs AM-259-M11 NODE 11 8", "AM-262-N05 contigs AM-262-N05 NODE 1 28", "AM-332-C20 contigs AM-332-C20 NODE 1 9", "AH-562-E04 contigs AH-562-E04 NODE 1 66", "AM-334-E18 contigs AM-334-E18 NODE 8 17", "AH-735-N13 contigs AH-735-N13 NODE 1 33", "AH-888-P11 contigs AH-888-P11 NODE 1 33", "AH-735-G19 contigs AH-735-G19 NODE 42 7", "AM-262-D11 contigs AM-262-D11 NODE 12 11", "AG-538-G20 contigs AG-538-G20 NODE 5 16", "AM-255-L07 contigs AM-255-L07 NODE 1 15", "AH-991-G23 contigs AH-991-G23 NODE 5 19", "AG-538-F05 contigs AG-538-F05 NODE 22 5", "AG-538-L09 contigs AG-538-L09 NODE 27 1", "AM-288-B14 contigs AM-288-B14 NODE 8 19", "AH-305-J14 contigs AH-305-J14 NODE 40 1", "AM-249-N23 contigs AM-249-N23 NODE 16 5", "AM-184-P16 contigs AM-184-P16 NODE 16 16", "AM-184-L15 contigs AM-184-L15 NODE 19 6", "AH-613-J18 contigs AH-613-J18 NODE 46 1", "AM-332-B03 contigs AM-332-B03 NODE 38 1", "AM-685-E15 contigs SCGC AM-685-E15 contig1 67", "AM-235-O15 contigs AM-235-O15 NODE 4 8", "AM-235-I03 contigs AM-235-I03 NODE 5 50", "AM-255-C22 contigs AM-255-C22 NODE 12 1", "AM-208-G02 contigs AM-208-G02 NODE 16 1", "AM-184-I13 contigs AM-184-I13 NODE 27 4", "AH-302-L19 contigs AH-302-L19 NODE 23 10", "AG-538-N07 contigs AG-538-N07 NODE 14 1", "AM-262-F15 contigs AM-262-F15 NODE 1 1", "AM-334-C08 contigs AM-334-C08 NODE 30 2", "AM-118-N17 contigs AM-118-N17 NODE 28 2", "AG-538-N13 contigs AG-538-N13 NODE 14 10", "AH-562-B18 contigs AH-562-B18 NODE 2 36", "AM-257-B10 contigs AM-257-B10 NODE 1 6", "AM-138-N14 contigs AM-138-N14 NODE 3 44", "AM-121-F09 contigs AM-121-F09 NODE 5 20", "AM-284-D11 contigs AM-284-D11 NODE 10 11", "AM-259-E13 contigs AM-259-E13 NODE 25 4", "AM-257-P09 contigs AM-257-P09 NODE 36 1", "AM-184-N06 contigs AM-184-N06 NODE 2 36", "AM-135-P16 contigs AM-135-P16 NODE 7 5", "AH-988-E18 contigs AH-988-E18 NODE 2 34", "AM-243-E13 contigs AM-243-E13 NODE 2 19", "AM-235-G18 contigs AM-235-G18 NODE 6 14", "AH-573-A04 contigs AH-573-A04 NODE 16 9", "AM-243-A06 contigs AM-243-A06 NODE 2 14", "AM-235-N15 contigs AM-235-N15 NODE 19 2", "AM-235-J04 contigs AM-235-J04 NODE 15 3", "AM-235-D15 contigs AM-235-D15 NODE 13 1", "AM-235-D14 contigs AM-235-D14 NODE 5 9", "AM-187-F11 contigs AM-187-F11 NODE 17 13", "AH-991-J22 contigs AH-991-J22 NODE 1 51", "AH-985-I18 contigs AH-985-I18 NODE 13 6", "AH-735-A03 contigs AH-735-A03 NODE 3 36", "AH-613-L15 contigs AH-613-L15 NODE 6 16", "AH-613-F05 contigs AH-613-F05 NODE 5 20", "AH-611-P06 contigs AH-611-P06 NODE 2 39", "AM-259-C23 contigs AM-259-C23 NODE 5 21", "AM-216-I04 contigs AM-216-I04 NODE 12 8", "AM-284-A18 contigs AM-284-A18 NODE 18 5", "AH-613-N14 contigs AH-613-N14 NODE 2 20", "AM-206-B22 contigs AM-206-B22 NODE 11 10", "AH-573-A17 contigs AH-573-A17 NODE 5 15", "AM-208-N16 contigs AM-208-N16 NODE 25 3", "AM-158-K17 contigs AM-158-K17 NODE 13 9", "AM-249-B18 contigs AM-249-B18 NODE 2 38", "AM-264-E15 contigs AM-264-E15 NODE 5 20", "AM-255-N22 contigs AM-255-N22 NODE 4 21", "AM-187-O15 contigs AM-187-O15 NODE 1 21", "AM-184-P10 contigs AM-184-P10 NODE 17 1", "AH-305-K19 contigs AH-305-K19 NODE 5 46", "AM-276-J10 contigs AM-276-J10 NODE 6 20", "AG-538-D22 contigs AG-538-D22 NODE 1 40", "AH-302-P11 contigs AH-302-P11 NODE 9 26", "AM-201-I18 contigs AM-201-I18 NODE 1 44", "AH-735-C17 contigs AH-735-C17 NODE 38 3", "AH-473-B13 contigs AH-473-B13 NODE 2 21", "AM-286-B19 contigs AM-286-B19 NODE 6 19", "AH-613-M11 contigs AH-613-M11 NODE 2 11", "AH-611-F04 contigs AH-611-F04 NODE 1 19", "AH-611-D02 contigs AH-611-D02 NODE 6 17", "AM-247-M19 contigs AM-247-M19 NODE 14 13", "AM-247-A03 contigs AM-247-A03 NODE 7 19", "AH-573-C14 contigs AH-573-C14 NODE 6 3", "AM-255-O14 contigs AM-255-O14 NODE 8 19", "AM-118-D04 contigs AM-118-D04 NODE 4 9", "AM-118-B18 contigs AM-118-B18 NODE 9 2", "AM-262-N08 contigs AM-262-N08 NODE 1 20", "AM-201-N07 contigs AM-201-N07 NODE 3 15", "AM-249-I18 contigs AM-249-I18 NODE 19 1", "AM-184-P01 contigs AM-184-P01 NODE 7 15", "AH-573-F17 contigs AH-573-F17 NODE 1 96", "AG-538-L06 contigs AG-538-L06 NODE 39 2", "AH-985-L08 contigs AH-985-L08 NODE 32 2", "AM-262-O18 contigs AM-262-O18 NODE 48 2", "AM-262-A08 contigs AM-262-A08 NODE 10 7", "AM-264-L18 contigs AM-264-L18 NODE 17 8", "AM-118-G18 contigs AM-118-G18 NODE 8 13", "AM-164-E11 contigs AM-164-E11 NODE 12 11", "AM-216-L04 contigs AM-216-L04 NODE 3 24", "AM-118-E23 contigs AM-118-E23 NODE 6 21", "AM-164-A09 contigs AM-164-A09 NODE 17 1", "AM-262-D04 contigs AM-262-D04 NODE 1 22", "AM-118-C11 contigs AM-118-C11 NODE 1 11", "AH-613-I15 contigs AH-613-I15 NODE 12 10", "AM-201-O16 contigs AM-201-O16 NODE 1 8", "AM-216-B20 contigs AM-216-B20 NODE 2 86", "AM-208-L15 contigs AM-208-L15 NODE 7 4", "AM-286-I23 contigs AM-286-I23 NODE 1 35", "AM-216-D08 contigs AM-216-D08 NODE 2 34", "AM-118-A07 contigs AM-118-A07 NODE 5 17", "AM-255-M06 contigs AM-255-M06 NODE 22 3", "AM-247-L03 contigs AM-247-L03 NODE 1 18", "AM-257-M14 contigs AM-257-M14 NODE 15 9", "AM-257-M14 contigs AM-257-M14 NODE 15 1", "AM-284-G14 contigs AM-284-G14 NODE 1 20", "AM-257-D05 contigs AM-257-D05 NODE 10 1", "AM-249-B17 contigs AM-249-B17 NODE 1 41", "AM-121-N03 contigs AM-121-N03 NODE 55 2", "AH-613-E07 contigs AH-613-E07 NODE 27 6", "AM-235-N11 contigs AM-235-N11 NODE 66 4", "AM-216-K08 contigs AM-216-K08 NODE 2 16", "AM-208-D10 contigs AM-208-D10 NODE 5 16", "AM-118-M15 contigs AM-118-M15 NODE 4 12", "AM-216-E08 contigs AM-216-E08 NODE 11 6", "AH-735-K15 contigs AH-735-K15 NODE 1 15", "AM-201-B16 contigs AM-201-B16 NODE 7 3", "AH-613-E20 contigs AH-613-E20 NODE 9 12", "AH-735-I22 contigs AH-735-I22 NODE 8 14", "AH-611-D06 contigs AH-611-D06 NODE 8 29", "AH-613-E11 contigs AH-613-E11 NODE 7 19", "AM-334-P15 contigs AM-334-P15 NODE 5 14", "AM-216-P16 contigs AM-216-P16 NODE 1 47", "AH-982-I09 contigs AH-982-I09 NODE 7 7", "AM-164-N21 contigs AM-164-N21 NODE 4 25", "AM-201-E11 contigs AM-201-E11 NODE 3 12", "AM-235-O03 contigs AM-235-O03 NODE 4 15", "AM-286-F02 contigs AM-286-F02 NODE 14 1", "AH-888-F10 contigs AH-888-F10 NODE 3 13", "AH-735-M10 contigs AH-735-M10 NODE 2 39", "AH-573-P17 contigs AH-573-P17 NODE 3 48", "AM-288-C17 contigs AM-288-C17 NODE 49 1", "AH-985-K15 contigs AH-985-K15 NODE 45 3", "AH-991-D11 contigs AH-991-D11 NODE 9 20", "AH-985-K08 contigs AH-985-K08 NODE 27 1", "AH-573-A18 contigs AH-573-A18 NODE 3 36", "AH-613-J20 contigs AH-613-J20 NODE 6 20", "AH-613-M03 contigs AH-613-M03 NODE 6 19", "AM-216-L05 contigs AM-216-L05 NODE 15 3", "AM-262-N22 contigs AM-262-N22 NODE 13 18", "AM-255-N03 contigs AM-255-N03 NODE 10 3", "AM-255-M18 contigs AM-255-M18 NODE 2 3", "AM-286-P13 contigs AM-286-P13 NODE 9 14", "AM-286-O15 contigs AM-286-O15 NODE 7 13", "AM-235-I09 contigs AM-235-I09 NODE 4 19", "AH-735-O14 contigs AH-735-O14 NODE 8 20", "AM-201-G23 contigs AM-201-G23 NODE 8 20", "AM-164-B09 contigs AM-164-B09 NODE 1 15", "AM-276-M20 contigs AM-276-M20 NODE 18 16", "AM-135-J18 contigs AM-135-J18 NODE 1 12", "AM-127-L08 contigs AM-127-L08 NODE 1 21", "AH-735-P04 contigs AH-735-P04 NODE 55 2", "AM-276-N02 contigs AM-276-N02 NODE 8 21", "AM-334-B22 contigs AM-334-B22 NODE 2 32", "AH-735-K11 contigs AH-735-K11 NODE 11 9", "AM-164-O15 contigs AM-164-O15 NODE 19 4", "AH-562-K21 contigs AH-562-K21 NODE 1 35", "AM-262-M09 contigs AM-262-M09 NODE 1 27", "AH-562-N11 contigs AH-562-N11 NODE 20 5", "AM-235-G06 contigs AM-235-G06 NODE 4 15", "AM-334-M08 contigs AM-334-M08 NODE 12 15", "AM-201-O10 contigs AM-201-O10 NODE 1 15", "AH-562-N17 contigs AH-562-N17 NODE 3 58", "AH-562-C22 contigs AH-562-C22 NODE 2 98", "AH-735-P08 contigs AH-735-P08 NODE 4 37", "AM-118-M18 contigs AM-118-M18 NODE 5 13", "AH-735-J03 contigs AH-735-J03 NODE 16 18", "AM-184-J15 contigs AM-184-J15 NODE 12 10", "AM-158-A06 contigs AM-158-A06 NODE 6 18", "AH-735-A10 contigs AH-735-A10 NODE 54 3", "AM-216-N08 contigs AM-216-N08 NODE 10 1", "AM-249-I23 contigs AM-249-I23 NODE 37 1", "AM-262-M03 contigs AM-262-M03 NODE 24 4", "AG-538-O07 contigs AG-538-O07 NODE 12 1", "AM-243-C08 contigs AM-243-C08 NODE 16 14", "AM-334-L14 contigs AM-334-L14 NODE 24 2", "AM-332-I06 contigs AM-332-I06 NODE 19 1", "AH-470-F08 contigs AH-470-F08 NODE 17 15", "AM-334-O04 contigs AM-334-O04 NODE 22 7", "AM-164-A01 contigs AM-164-A01 NODE 17 1", "AM-216-F22 contigs AM-216-F22 NODE 6 1", "AM-201-C21 contigs AM-201-C21 NODE 1 14", "AM-118-I08 contigs AM-118-I08 NODE 6 1", "AM-118-G08 contigs AM-118-G08 NODE 20 5", "AH-562-M22 contigs AH-562-M22 NODE 8 7", "AM-216-C16 contigs AM-216-C16 NODE 6 20", "AM-118-E14 contigs AM-118-E14 NODE 1 43", "AH-735-M08 contigs AH-735-M08 NODE 17 14", "AH-735-G03 contigs AH-735-G03 NODE 2 65", "AM-334-K03 contigs AM-334-K03 NODE 1 92", "AM-201-F22 contigs AM-201-F22 NODE 11 6", "AM-201-A14 contigs AM-201-A14 NODE 8 14", "AH-735-G22 contigs AH-735-G22 NODE 5 31", "AM-216-M21 contigs AM-216-M21 NODE 21 16", "AM-286-N18 contigs AM-286-N18 NODE 8 18", "AM-201-K20 contigs AM-201-K20 NODE 9 10", "AH-735-L20 contigs AH-735-L20 NODE 4 10", "AH-735-G21 contigs AH-735-G21 NODE 5 19", "AM-201-A20 contigs AM-201-A20 NODE 4 18", "AH-888-C20 contigs AH-888-C20 NODE 12 2", "AM-255-O23 contigs AM-255-O23 NODE 4 4", "AM-201-K09 contigs AM-201-K09 NODE 2 3", "AH-735-O16 contigs AH-735-O16 NODE 6 12", "AM-164-N16 contigs AM-164-N16 NODE 9 3", "AM-138-J05 contigs AM-138-J05 NODE 7 11", "AH-735-E08 contigs AH-735-E08 NODE 2 13", "AM-255-C06 contigs AM-255-C06 NODE 5 3", "AM-118-P07 contigs AM-118-P07 NODE 8 13", "AM-216-G06 contigs AM-216-G06 NODE 9 11", "AM-201-L16 contigs AM-201-L16 NODE 2 49", "AM-201-G09 contigs AM-201-G09 NODE 18 6", "AM-118-K06 contigs AM-118-K06 NODE 7 15", "AM-118-J17 contigs AM-118-J17 NODE 22 6", "AH-735-D20 contigs AH-735-D20 NODE 1 15", "AH-305-P08 contigs AH-305-P08 NODE 1 73", "AM-138-C08 contigs AM-138-C08 NODE 4 20", "AM-121-B14 contigs AM-121-B14 NODE 2 19", "AM-262-C20 contigs AM-262-C20 NODE 1 12", "AH-735-A14 contigs AH-735-A14 NODE 1 14", "AH-562-C02 contigs AH-562-C02 NODE 2 110", "AM-118-N05 contigs AM-118-N05 NODE 6 8", "AM-216-A17 contigs AM-216-A17 NODE 6 4", "AM-201-F23 contigs AM-201-F23 NODE 2 35", "AH-735-A06 contigs AH-735-A06 NODE 8 11", "AM-262-P21 contigs AM-262-P21 NODE 2 36", "AM-216-D21 contigs AM-216-D21 NODE 16 13", "AG-538-B18 contigs AG-538-B18 NODE 3 16", "AH-613-K13 contigs AH-613-K13 NODE 1 11", "AH-985-B06 contigs AH-985-B06 NODE 23 3", "AH-991-A15 contigs AH-991-A15 NODE 14 25", "AH-470-C06 contigs AH-470-C06 NODE 11 9", "AH-735-I07 contigs AH-735-I07 NODE 15 3", "AM-138-N21 contigs AM-138-N21 NODE 7 12", "AH-735-N19 contigs AH-735-N19 NODE 4 39", "AM-164-B17 contigs AM-164-B17 NODE 25 5", "AM-164-F23 contigs AM-164-F23 NODE 2 19", "AH-735-E02 contigs AH-735-E02 NODE 7 21", "AM-118-C14 contigs AM-118-C14 NODE 5 12", "AH-735-J22 contigs AH-735-J22 NODE 4 39", "AM-118-O22 contigs AM-118-O22 NODE 33 2", "AM-118-D23 contigs AM-118-D23 NODE 15 2", "AH-735-P06 contigs AH-735-P06 NODE 12 3", "AH-735-K21 contigs AH-735-K21 NODE 9 8", "AH-735-F16 contigs AH-735-F16 NODE 10 24", "AH-735-E17 contigs AH-735-E17 NODE 6 35", "AH-735-A17 contigs AH-735-A17 NODE 1 21"]


### OUTPUTS #####
DIR_output_1 = "./1_output_simplified_alignment_for_rhods_to_visually_screen/"
PATH_output_1a = DIR_output_1 + "461_2Fe2S_rhodopsins_that_might_have_helix.fasta"
PATH_output_1b = DIR_output_1 + "141_other_rhodopsins_that_might_have_helix.fasta" 

#### OVERVIEW #####
print("OVERVIEW\n")
print("I have visually inspected the alignment "+PATH_input_1)
print("And found " + str(len(LIST_IDs_alignable_8th_helix)) + ' GORG Dark rhodopsins that have alignable AAs possibly forming an N-terminal 8th helix')
print("Part I of this notebook will subset that alignment into two output fastas containing.\n\t(1)The 2Fe2S rhodopsins\n\t(2)The Non2Fe2S rhodopsins")
print("For further visual analysis. I.e. So that I can group them by seq similarity and then visually evaluate one rep of each group in alphafold.\n")

print("\n\nPART I\n")
print(str(len(LIST_2Fe2S_rhodopsins)) + ' PROTEOrhodopsins phylogenetically branching within 2Fe2S rhodopsins')

LIST_non2Fe2S_to_check_for_8th_helix = [i for i in LIST_IDs_alignable_8th_helix if i not in LIST_2Fe2S_rhodopsins]

### Have to do some seqID reformatting since iToL vs. the AA alignment have different format
### SeqIDs as formatted in Tianyi's AA alignment
LIST_full_seqIDs = ["AM-388-N16_contigs_AM-388-N16_NODE_36_15_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-D08_contigs_AM-388-D08_NODE_84_2_rank:_D_Heliorhodopsin_PF18761.4", "AH-985-B07_contigs_AH-985-B07_NODE_9_10_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-E07_contigs_AH-470-E07_NODE_2_54_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-276-K05_contigs_AM-276-K05_NODE_11_24_rank:_C_sensory_rhodopsin", "AM-332-O14_contigs_AM-332-O14_NODE_1_129_rank:_C_sensory_rhodopsin", "AM-247-G17_contigs_AM-247-G17_NODE_32_19_rank:_C_sensory_rhodopsin", "AH-473-N09_contigs_AH-473-N09_NODE_54_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-C16_contigs_AM-332-C16_NODE_8_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-M03_contigs_AM-332-M03_NODE_38_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-E21_contigs_AM-332-E21_NODE_15_18_rank:_C_sensory_rhodopsin", "AM-332-J04_contigs_AM-332-J04_NODE_8_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-N05_contigs_AM-332-N05_NODE_2_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-D04_contigs_AH-985-D04_NODE_22_2_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-F20_contigs_AM-332-F20_NODE_43_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-L13_contigs_AM-332-L13_NODE_52_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-288-O17_contigs_AM-288-O17_NODE_6_28_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-C03_contigs_AM-332-C03_NODE_18_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-K10_contigs_AM-332-K10_NODE_9_22_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-K13_contigs_AM-332-K13_NODE_2_2_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-L18_contigs_AH-470-L18_NODE_26_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-982-J09_contigs_AH-982-J09_NODE_10_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-982-L18_contigs_AH-982-L18_NODE_12_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-E13_contigs_AH-985-E13_NODE_1_133_rank:_C_sensory_rhodopsin", "AH-985-J19_contigs_AH-985-J19_NODE_2_49_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-E15_contigs_AM-332-E15_NODE_10_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-276-P03_contigs_AM-276-P03_NODE_2_149_rank:_C_sensory_rhodopsin", "AM-332-L06_contigs_AM-332-L06_NODE_14_2_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-C14_contigs_AM-332-C14_NODE_11_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-P08_contigs_AM-332-P08_NODE_42_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-K08_contigs_AM-332-K08_NODE_18_22_rank:_C_sensory_rhodopsin", "AM-332-E05_contigs_AM-332-E05_NODE_4_106_rank:_C_sensory_rhodopsin", "AM-685-E19_contigs_SCGC_AM-685-E19_contig6_29_rank:_C_sensory_rhodopsin", "AH-302-L09_contigs_AH-302-L09_NODE_11_3_rank:_D_Bacteriorhodopsin-like", "AH-305-A06_contigs_AH-305-A06_NODE_1_19_rank:_D_Bacteriorhodopsin-like", "AH-305-M13_contigs_AH-305-M13_NODE_9_6_rank:_D_Bacteriorhodopsin-like", "AH-307-I04_contigs_AH-307-I04_NODE_6_11_rank:_D_Bacteriorhodopsin-like", "AG-538-G09_contigs_AG-538-G09_NODE_17_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-309-A13_contigs_AH-309-A13_NODE_23_14_rank:_D_Bacteriorhodopsin-like", "AM-124-I15_contigs_AM-124-I15_NODE_53_2_rank:_D_Bacteriorhodopsin-like", "AM-276-A01_contigs_AM-276-A01_NODE_21_6_rank:_D_Bacteriorhodopsin-like", "AM-229-K19_contigs_AM-229-K19_NODE_37_2_rank:_D_Bacteriorhodopsin-like", "AM-124-E05_contigs_AM-124-E05_NODE_5_31_rank:_D_Bacteriorhodopsin-like", "AM-161-G23_contigs_AM-161-G23_NODE_14_19_rank:_D_Bacteriorhodopsin-like", "AM-276-F23_contigs_AM-276-F23_NODE_5_6_rank:_D_Bacteriorhodopsin-like", "AM-127-K13_contigs_AM-127-K13_NODE_40_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-243-J14_contigs_AM-243-J14_NODE_4_6_rank:_D_Bacteriorhodopsin-like", "AM-249-N20_contigs_AM-249-N20_NODE_7_21_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-573-D18_contigs_AH-573-D18_NODE_1_7_rank:_D_Bacteriorhodopsin-like", "AH-307-G21_contigs_AH-307-G21_NODE_16_15_rank:_C_sensory_rhodopsin", "AM-210-D16_contigs_AM-210-D16_NODE_2_12_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-124-C13_contigs_AM-124-C13_NODE_1_53_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-135-L13_contigs_AM-135-L13_NODE_3_34_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-184-E04_contigs_AM-184-E04_NODE_2_34_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-276-A04_contigs_AM-276-A04_NODE_3_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-280-L17_contigs_AM-280-L17_NODE_1_29_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-161-A19_contigs_AM-161-A19_NODE_39_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-243-K07_contigs_AM-243-K07_NODE_45_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-284-B07_contigs_AM-284-B07_NODE_1_42_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-158-K11_contigs_AM-158-K11_NODE_35_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-305-K23_contigs_AH-305-K23_NODE_9_11_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-255-O11_contigs_AM-255-O11_NODE_2_57_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-118-P09_contigs_AM-118-P09_NODE_6_25_rank:_D_Bacteriorhodopsin-like", "AM-334-O09_contigs_AM-334-O09_NODE_4_24_rank:_D_Bacteriorhodopsin-like", "AM-124-G09_contigs_AM-124-G09_NODE_36_11_rank:_C_sensory_rhodopsin", "AM-280-G11_contigs_AM-280-G11_NODE_31_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-249-M11_contigs_AM-249-M11_NODE_6_11_rank:_D_Bacteriorhodopsin-like", "AM-288-F06_contigs_AM-288-F06_NODE_16_8_rank:_D_Bacteriorhodopsin-like", "AM-255-C18_contigs_AM-255-C18_NODE_18_7_rank:_D_Bacteriorhodopsin-like", "AH-573-A09_contigs_AH-573-A09_NODE_10_29_rank:_C_sensory_rhodopsin", "AH-573-O15_contigs_AH-573-O15_NODE_7_18_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-184-P16_contigs_AM-184-P16_NODE_22_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-264-F14_contigs_AM-264-F14_NODE_42_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-988-B10_contigs_AH-988-B10_NODE_8_10_rank:_D_Bacteriorhodopsin-like", "AM-124-F21_contigs_AM-124-F21_NODE_37_3_rank:_D_Bacteriorhodopsin-like", "AM-135-M08_contigs_AM-135-M08_NODE_13_12_rank:_D_Bacteriorhodopsin-like", "AM-121-N16_contigs_AM-121-N16_NODE_23_6_rank:_D_Bacteriorhodopsin-like", "AM-280-K21_contigs_AM-280-K21_NODE_13_19_rank:_D_Bacteriorhodopsin-like", "AM-118-K03_contigs_AM-118-K03_NODE_4_20_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-229-F19_contigs_AM-229-F19_NODE_1_60_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-121-I21_contigs_AM-121-I21_NODE_2_56_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-208-C11_contigs_AM-208-C11_NODE_8_13_rank:_D_Bacteriorhodopsin-like", "AM-184-L15_contigs_AM-184-L15_NODE_1_56_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-O05_contigs_AM-187-O05_NODE_6_18_rank:_D_Bacteriorhodopsin-like", "AM-118-K21_contigs_AM-118-K21_NODE_29_2_rank:_D_Bacteriorhodopsin-like", "AM-161-C03_contigs_AM-161-C03_NODE_5_16_rank:_D_Bacteriorhodopsin-like", "AM-184-I06_contigs_AM-184-I06_NODE_7_1_rank:_D_Bacteriorhodopsin-like", "AM-121-P05_contigs_AM-121-P05_NODE_2_82_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-255-M09_contigs_AM-255-M09_NODE_6_19_rank:_D_Bacteriorhodopsin-like", "AH-573-M15_contigs_AH-573-M15_NODE_20_5_rank:_D_Bacteriorhodopsin-like", "AH-988-C21_contigs_AH-988-C21_NODE_8_18_rank:_D_Bacteriorhodopsin-like", "AM-208-K18_contigs_AM-208-K18_NODE_9_14_rank:_D_Bacteriorhodopsin-like", "AM-210-J07_contigs_AM-210-J07_NODE_1_73_rank:_D_Bacteriorhodopsin-like", "AM-187-J02_contigs_AM-187-J02_NODE_11_11_rank:_C_sensory_rhodopsin", "AM-249-B14_contigs_AM-249-B14_NODE_11_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-C08_contigs_AM-332-C08_NODE_8_13_rank:_D_Bacteriorhodopsin-like", "AM-243-C16_contigs_AM-243-C16_NODE_23_6_rank:_D_Bacteriorhodopsin-like", "AM-288-B14_contigs_AM-288-B14_NODE_1_73_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-118-K20_contigs_AM-118-K20_NODE_12_12_rank:_C_sensory_rhodopsin", "AM-264-E23_contigs_AM-264-E23_NODE_9_12_rank:_D_Bacteriorhodopsin-like", "AM-121-E03_contigs_AM-121-E03_NODE_32_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-280-C14_contigs_AM-280-C14_NODE_12_13_rank:_D_Bacteriorhodopsin-like", "AM-334-G07_contigs_AM-334-G07_NODE_8_4_rank:_D_Bacteriorhodopsin-like", "AM-334-F08_contigs_AM-334-F08_NODE_16_12_rank:_D_Bacteriorhodopsin-like", "AM-249-M09_contigs_AM-249-M09_NODE_76_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-118-G13_contigs_AM-118-G13_NODE_18_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-184-M06_contigs_AM-184-M06_NODE_3_16_rank:_D_Bacteriorhodopsin-like", "AG-538-D17_contigs_AG-538-D17_NODE_27_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-118-P17_contigs_AM-118-P17_NODE_3_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-249-K22_contigs_AM-249-K22_NODE_1_30_rank:_D_Bacteriorhodopsin-like", "AM-262-E15_contigs_AM-262-E15_NODE_12_11_rank:_C_sensory_rhodopsin", "AM-332-G04_contigs_AM-332-G04_NODE_9_17_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-216-B07_contigs_AM-216-B07_NODE_17_7_rank:_D_Bacteriorhodopsin-like", "AH-573-P04_contigs_AH-573-P04_NODE_2_36_rank:_D_Bacteriorhodopsin-like", "AM-187-C18_contigs_AM-187-C18_NODE_24_8_rank:_D_Bacteriorhodopsin-like", "AH-611-A20_contigs_AH-611-A20_NODE_13_33_rank:_C_sensory_rhodopsin", "AM-216-K15_contigs_AM-216-K15_NODE_2_34_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-D04_contigs_AM-334-D04_NODE_17_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-288-P10_contigs_AM-288-P10_NODE_5_23_rank:_D_Bacteriorhodopsin-like", "AH-562-L13_contigs_AH-562-L13_NODE_4_53_rank:_D_Bacteriorhodopsin-like", "AM-208-K04_contigs_AM-208-K04_NODE_11_1_rank:_D_Bacteriorhodopsin-like", "AH-305-K03_contigs_AH-305-K03_NODE_1_39_rank:_D_Bacteriorhodopsin-like", "AM-216-D10_contigs_AM-216-D10_NODE_3_6_rank:_D_Bacteriorhodopsin-like", "AH-735-B03_contigs_AH-735-B03_NODE_1_75_rank:_D_Bacteriorhodopsin-like", "AM-216-C21_contigs_AM-216-C21_NODE_24_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-225-L16_contigs_AM-225-L16_NODE_28_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-216-J22_contigs_AM-216-J22_NODE_18_5_rank:_D_Bacteriorhodopsin-like", "AM-208-C02_contigs_AM-208-C02_NODE_49_1_rank:_D_Bacteriorhodopsin-like", "AM-262-I09_contigs_AM-262-I09_NODE_1_1_rank:_D_Bacteriorhodopsin-like", "AH-573-C21_contigs_AH-573-C21_NODE_17_3_rank:_D_Bacteriorhodopsin-like", "AH-573-F17_contigs_AH-573-F17_NODE_1_133_rank:_D_Bacteriorhodopsin-like", "AH-735-G09_contigs_AH-735-G09_NODE_22_8_rank:_D_Bacteriorhodopsin-like", "AH-888-M17_contigs_AH-888-M17_NODE_10_4_rank:_D_Bacteriorhodopsin-like", "AM-286-I22_contigs_AM-286-I22_NODE_19_7_rank:_D_Bacteriorhodopsin-like", "AH-735-J03_contigs_AH-735-J03_NODE_15_22_rank:_D_Bacteriorhodopsin-like", "AM-121-O09_contigs_AM-121-O09_NODE_22_3_rank:_D_Bacteriorhodopsin-like", "AM-164-A01_contigs_AM-164-A01_NODE_7_4_rank:_D_Bacteriorhodopsin-like", "AM-187-K07_contigs_AM-187-K07_NODE_20_8_rank:_D_Bacteriorhodopsin-like", "AH-985-K08_contigs_AH-985-K08_NODE_4_21_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-208-D10_contigs_AM-208-D10_NODE_22_11_rank:_D_Bacteriorhodopsin-like", "AM-118-M15_contigs_AM-118-M15_NODE_7_18_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-I09_contigs_AM-235-I09_NODE_20_3_rank:_D_Bacteriorhodopsin-like", "AM-247-A03_contigs_AM-247-A03_NODE_6_10_rank:_D_Bacteriorhodopsin-like", "AM-249-G22_contigs_AM-249-G22_NODE_9_1_rank:_D_Bacteriorhodopsin-like", "AM-235-K05_contigs_AM-235-K05_NODE_26_4_rank:_D_Bacteriorhodopsin-like", "AM-249-I18_contigs_AM-249-I18_NODE_25_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-286-O15_contigs_AM-286-O15_NODE_9_11_rank:_D_Bacteriorhodopsin-like", "AM-138-C10_contigs_AM-138-C10_NODE_30_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-573-A18_contigs_AH-573-A18_NODE_23_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-J09_contigs_AH-613-J09_NODE_12_21_rank:_C_sensory_rhodopsin", "AH-888-G18_contigs_AH-888-G18_NODE_18_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-N06_contigs_AH-735-N06_NODE_20_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-I06_contigs_AM-332-I06_NODE_5_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-991-G23_contigs_AH-991-G23_NODE_11_2_rank:_D_Bacteriorhodopsin-like", "AM-255-N03_contigs_AM-255-N03_NODE_25_9_rank:_D_Bacteriorhodopsin-like", "AM-201-C21_contigs_AM-201-C21_NODE_36_6_rank:_D_Bacteriorhodopsin-like", "AM-118-M18_contigs_AM-118-M18_NODE_11_14_rank:_D_Bacteriorhodopsin-like", "AM-255-M18_contigs_AM-255-M18_NODE_31_6_rank:_D_Bacteriorhodopsin-like", "AH-611-F04_contigs_AH-611-F04_NODE_13_7_rank:_D_Bacteriorhodopsin-like", "AH-473-B13_contigs_AH-473-B13_NODE_22_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-E15_contigs_AM-187-E15_NODE_11_21_rank:_D_Bacteriorhodopsin-like", "AM-216-C16_contigs_AM-216-C16_NODE_7_20_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-P04_contigs_AH-735-P04_NODE_60_1_rank:_D_Bacteriorhodopsin-like", "AH-991-F05_contigs_AH-991-F05_NODE_29_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-201-G09_contigs_AM-201-G09_NODE_8_11_rank:_D_Bacteriorhodopsin-like", "AM-255-P07_contigs_AM-255-P07_NODE_53_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-264-L18_contigs_AM-264-L18_NODE_9_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-164-B09_contigs_AM-164-B09_NODE_24_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-201-G23_contigs_AM-201-G23_NODE_4_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-302-C16_contigs_AH-302-C16_NODE_2_56_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-988-D21_contigs_AH-988-D21_NODE_11_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-158-K17_contigs_AM-158-K17_NODE_3_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-611-L14_contigs_AH-611-L14_NODE_5_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-A01_contigs_AH-735-A01_NODE_12_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-118-A20_contigs_AM-118-A20_NODE_46_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-280-E05_contigs_AM-280-E05_NODE_9_7_rank:_D_Bacteriorhodopsin-like", "AH-735-M18_contigs_AH-735-M18_NODE_1_10_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-573-N23_contigs_AH-573-N23_NODE_12_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-685-G04_contigs_SCGC_AM-685-G04_contig7_1_rank:_D_Bacteriorhodopsin-like", "AM-235-J07_contigs_AM-235-J07_NODE_9_13_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-888-O18_contigs_AH-888-O18_NODE_13_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-611-D02_contigs_AH-611-D02_NODE_22_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-262-N05_contigs_AM-262-N05_NODE_6_1_rank:_D_Bacteriorhodopsin-like", "AH-888-D11_contigs_AH-888-D11_NODE_14_1_rank:_D_Bacteriorhodopsin-like", "AM-247-K15_contigs_AM-247-K15_NODE_5_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-B22_contigs_AM-334-B22_NODE_3_22_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-138-J05_contigs_AM-138-J05_NODE_14_7_rank:_D_Bacteriorhodopsin-like", "AM-201-O16_contigs_AM-201-O16_NODE_13_10_rank:_C_sensory_rhodopsin", "AM-138-J05_contigs_AM-138-J05_NODE_14_13_rank:_C_sensory_rhodopsin", "AH-735-E08_contigs_AH-735-E08_NODE_3_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-I15_contigs_AH-613-I15_NODE_8_11_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-288-J07_contigs_AM-288-J07_NODE_3_47_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-286-B19_contigs_AM-286-B19_NODE_9_12_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-G22_contigs_AH-735-G22_NODE_9_10_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-O18_contigs_AM-334-O18_NODE_36_6_rank:_D_Bacteriorhodopsin-like", "AG-538-B02_contigs_AG-538-B02_NODE_14_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-118-J17_contigs_AM-118-J17_NODE_11_23_rank:_C_sensory_rhodopsin", "AH-735-M08_contigs_AH-735-M08_NODE_19_10_rank:_C_sensory_rhodopsin", "AH-735-E21_contigs_AH-735-E21_NODE_15_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-208-L13_contigs_AM-208-L13_NODE_13_10_rank:_D_Bacteriorhodopsin-like", "AH-305-P08_contigs_AH-305-P08_NODE_2_27_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-259-G09_contigs_AM-259-G09_NODE_1_27_rank:_D_Bacteriorhodopsin-like", "AM-276-M20_contigs_AM-276-M20_NODE_23_17_rank:_D_Bacteriorhodopsin-like", "AM-280-J04_contigs_AM-280-J04_NODE_36_1_rank:_D_Bacteriorhodopsin-like", "AM-127-L08_contigs_AM-127-L08_NODE_54_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-249-I23_contigs_AM-249-I23_NODE_4_13_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-276-N02_contigs_AM-276-N02_NODE_3_36_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-302-P11_contigs_AH-302-P11_NODE_6_24_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-B08_contigs_AM-235-B08_NODE_6_15_rank:_D_Bacteriorhodopsin-like", "AM-288-P17_contigs_AM-288-P17_NODE_2_39_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-L17_contigs_AM-332-L17_NODE_8_10_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-C06_contigs_AM-334-C06_NODE_11_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-N13_contigs_AM-332-N13_NODE_3_11_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-A16_contigs_AM-332-A16_NODE_16_2_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-B10_contigs_AM-332-B10_NODE_8_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-N13_contigs_AH-735-N13_NODE_29_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-888-L04_contigs_AH-888-L04_NODE_15_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-247-J07_contigs_AM-247-J07_NODE_30_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-G10_contigs_AM-332-G10_NODE_17_14_rank:_D_Bacteriorhodopsin-like", "AH-305-D13_contigs_AH-305-D13_NODE_54_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-130-K09_contigs_AM-130-K09_NODE_64_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-P15_contigs_AH-470-P15_NODE_3_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-P04_contigs_AM-332-P04_NODE_9_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-C06_contigs_AM-332-C06_NODE_4_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-F18_contigs_AM-332-F18_NODE_2_10_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-A17_contigs_AH-985-A17_NODE_1_74_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-991-E06_contigs_AH-991-E06_NODE_9_14_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-288-B03_contigs_AM-288-B03_NODE_7_10_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-611-C05_contigs_AH-611-C05_NODE_43_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-A20_contigs_AH-470-A20_NODE_34_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-611-N23_contigs_AH-611-N23_NODE_2_61_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-E07_contigs_AH-985-E07_NODE_5_33_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-468-K22_contigs_AH-468-K22_NODE_12_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-C17_contigs_AH-470-C17_NODE_26_12_rank:_C_sensory_rhodopsin", "AH-470-J19_contigs_AH-470-J19_NODE_1_19_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-O19_contigs_AH-470-O19_NODE_17_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-A19_contigs_AH-473-A19_NODE_2_13_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-982-I21_contigs_AH-982-I21_NODE_1_111_rank:_C_sensory_rhodopsin", "AH-991-L03_contigs_AH-991-L03_NODE_9_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-D23_contigs_AM-235-D23_NODE_45_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-F04_contigs_AH-473-F04_NODE_4_28_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-J16_contigs_AH-473-J16_NODE_17_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-O05_contigs_AH-473-O05_NODE_1_56_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-P11_contigs_AH-470-P11_NODE_6_45_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-982-G22_contigs_AH-982-G22_NODE_2_170_rank:_C_sensory_rhodopsin", "AH-982-C03_contigs_AH-982-C03_NODE_1_59_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-F08_contigs_AH-470-F08_NODE_4_18_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-I07_contigs_AH-473-I07_NODE_7_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-N15_contigs_AH-470-N15_NODE_3_14_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-D09_contigs_AH-473-D09_NODE_22_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-B18_contigs_AH-985-B18_NODE_9_37_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-N19_contigs_AH-473-N19_NODE_7_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-O05_contigs_AH-985-O05_NODE_3_98_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-F18_contigs_AH-473-F18_NODE_5_35_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-J14_contigs_AH-473-J14_NODE_2_22_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-E22_contigs_AH-985-E22_NODE_5_10_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-A22_contigs_AM-332-A22_NODE_1_15_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-M16_contigs_AM-332-M16_NODE_6_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-D02_contigs_AM-332-D02_NODE_29_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-D03_contigs_AM-332-D03_NODE_11_29_rank:_C_sensory_rhodopsin", "AM-332-F21_contigs_AM-332-F21_NODE_6_37_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-L08_contigs_AM-332-L08_NODE_2_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-A01_contigs_AM-334-A01_NODE_10_10_rank:_C_sensory_rhodopsin", "AM-332-N08_contigs_AM-332-N08_NODE_21_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-N11_contigs_AM-334-N11_NODE_1_11_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-K23_contigs_AM-235-K23_NODE_3_24_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-C07_contigs_AH-473-C07_NODE_5_25_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-611-I23_contigs_AH-611-I23_NODE_4_21_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-611-P07_contigs_AH-611-P07_NODE_10_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-M17_contigs_AM-235-M17_NODE_15_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-611-O09_contigs_AH-611-O09_NODE_4_21_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-A09_contigs_AM-235-A09_NODE_37_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-164-N21_contigs_AM-164-N21_NODE_43_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-I05_contigs_AH-470-I05_NODE_3_24_rank:_C_sensory_rhodopsin_(db=kegg)", "AG-538-J23_contigs_AG-538-J23_NODE_9_13_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-G11_contigs_AM-187-G11_NODE_6_31_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-E11_contigs_AH-613-E11_NODE_44_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-E20_contigs_AH-613-E20_NODE_13_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-247-E21_contigs_AM-247-E21_NODE_2_64_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-P09_contigs_AM-187-P09_NODE_2_10_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-J20_contigs_AM-187-J20_NODE_47_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-225-A20_contigs_AM-225-A20_NODE_2_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-225-M07_contigs_AM-225-M07_NODE_45_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-201-A14_contigs_AM-201-A14_NODE_20_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-164-D11_contigs_AM-164-D11_NODE_5_20_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-M02_contigs_AM-334-M02_NODE_10_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-O04_contigs_AM-334-O04_NODE_7_15_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-I22_contigs_AH-735-I22_NODE_12_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-685-F13_contigs_SCGC_AM-685-F13_contig16_8_rank:_C_sensory_rhodopsin", "AM-332-C20_contigs_AM-332-C20_NODE_3_54_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-F09_contigs_AH-613-F09_NODE_36_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-D18_contigs_AH-985-D18_NODE_3_60_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-286-G07_contigs_AM-286-G07_NODE_1_25_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-184-C05_contigs_AM-184-C05_NODE_7_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-130-G06_contigs_AM-130-G06_NODE_6_65_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-259-O02_contigs_AM-259-O02_NODE_1_31_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-B18_contigs_AM-332-B18_NODE_12_2_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-N20_contigs_AM-332-N20_NODE_1_20_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-164-L02_contigs_AM-164-L02_NODE_1_29_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-243-A07_contigs_AM-243-A07_NODE_5_36_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-I03_contigs_AM-235-I03_NODE_31_2_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-E18_contigs_AM-334-E18_NODE_1_26_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-E07_contigs_AH-613-E07_NODE_18_12_rank:_C_sensory_rhodopsin", "AH-735-K15_contigs_AH-735-K15_NODE_2_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-216-E08_contigs_AM-216-E08_NODE_31_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-991-K03_contigs_AH-991-K03_NODE_2_46_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-201-N19_contigs_AM-201-N19_NODE_5_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-G21_contigs_AM-235-G21_NODE_12_20_rank:_C_sensory_rhodopsin", "AM-332-K06_contigs_AM-332-K06_NODE_3_14_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-D08_contigs_AM-332-D08_NODE_14_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-I16_contigs_AM-332-I16_NODE_4_12_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-G22_contigs_AM-332-G22_NODE_2_60_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-216-G06_contigs_AM-216-G06_NODE_2_33_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-262-C09_contigs_AM-262-C09_NODE_10_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-201-K09_contigs_AM-201-K09_NODE_20_2_rank:_D_Bacteriorhodopsin-like", "AM-206-N08_contigs_AM-206-N08_NODE_12_23_rank:_D_Bacteriorhodopsin-like", "AM-286-F07_contigs_AM-286-F07_NODE_1_59_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-286-K06_contigs_AM-286-K06_NODE_1_31_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-G21_contigs_AH-735-G21_NODE_2_64_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-M23_contigs_AH-735-M23_NODE_2_22_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-888-L22_contigs_AH-888-L22_NODE_6_17_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-I18_contigs_AH-985-I18_NODE_3_27_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-L08_contigs_AM-235-L08_NODE_7_29_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-D14_contigs_AM-235-D14_NODE_4_11_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-288-J17_contigs_AM-288-J17_NODE_1_17_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-249-B18_contigs_AM-249-B18_NODE_8_20_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-D15_contigs_AM-235-D15_NODE_6_13_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-243-E13_contigs_AM-243-E13_NODE_13_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-118-B02_contigs_AM-118-B02_NODE_29_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-N10_contigs_AH-613-N10_NODE_74_2_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-243-C08_contigs_AM-243-C08_NODE_4_23_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-229-O19_contigs_AM-229-O19_NODE_1_66_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-888-C20_contigs_AH-888-C20_NODE_1_33_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-573-J17_contigs_AH-573-J17_NODE_13_27_rank:_C_sensory_rhodopsin", "AH-611-P06_contigs_AH-611-P06_NODE_3_17_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-C13_contigs_AH-613-C13_NODE_7_13_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-C09_contigs_AM-235-C09_NODE_40_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-685-E15_contigs_SCGC_AM-685-E15_contig5_25_rank:_C_sensory_rhodopsin", "AH-735-O16_contigs_AH-735-O16_NODE_1_23_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-O15_contigs_AM-235-O15_NODE_1_24_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-685-F04_contigs_SCGC_AM-685-F04_contig7_14_rank:_C_sensory_rhodopsin", "AH-613-J18_contigs_AH-613-J18_NODE_9_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-J04_contigs_AM-235-J04_NODE_59_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-243-A06_contigs_AM-243-A06_NODE_1_62_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-B23_contigs_AM-187-B23_NODE_23_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-P18_contigs_AM-235-P18_NODE_1_65_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-A03_contigs_AH-735-A03_NODE_13_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-K08_contigs_AM-187-K08_NODE_6_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-888-O06_contigs_AH-888-O06_NODE_1_82_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-243-F23_contigs_AM-243-F23_NODE_3_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-247-K09_contigs_AM-247-K09_NODE_18_15_rank:_C_sensory_rhodopsin", "AM-286-N18_contigs_AM-286-N18_NODE_4_34_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-M17_contigs_AM-187-M17_NODE_7_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-A10_contigs_AH-735-A10_NODE_2_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-247-P05_contigs_AM-247-P05_NODE_1_23_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-243-B08_contigs_AM-243-B08_NODE_21_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-L16_contigs_AH-735-L16_NODE_2_28_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-991-J22_contigs_AH-991-J22_NODE_3_13_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-F14_contigs_AM-235-F14_NODE_2_22_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-L20_contigs_AH-735-L20_NODE_12_19_rank:_C_sensory_rhodopsin", "AM-184-O13_contigs_AM-184-O13_NODE_1_26_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-F11_contigs_AM-187-F11_NODE_3_15_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-689-N03_contigs_SCGC_AM-689-N03_contig1_25_rank:_C_sensory_rhodopsin", "AM-164-M18_contigs_AM-164-M18_NODE_6_28_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-I09_contigs_AH-613-I09_NODE_4_17_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-991-D11_contigs_AH-991-D11_NODE_21_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-286-I03_contigs_AM-286-I03_NODE_12_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-216-K08_contigs_AM-216-K08_NODE_54_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-D20_contigs_AH-735-D20_NODE_20_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-130-E23_contigs_AM-130-E23_NODE_6_20_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-210-A17_contigs_AM-210-A17_NODE_12_15_rank:_C_sensory_rhodopsin", "AM-243-F04_contigs_AM-243-F04_NODE_15_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-284-A18_contigs_AM-284-A18_NODE_43_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-B19_contigs_AM-334-B19_NODE_15_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-118-E14_contigs_AM-118-E14_NODE_6_12_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-611-J16_contigs_AH-611-J16_NODE_2_19_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-K19_contigs_AH-735-K19_NODE_5_13_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-208-K11_contigs_AM-208-K11_NODE_27_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-247-B14_contigs_AM-247-B14_NODE_1_43_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-K11_contigs_AH-735-K11_NODE_4_13_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-138-J06_contigs_AM-138-J06_NODE_3_17_rank:_D_Bacteriorhodopsin-like", "AG-538-M09_contigs_AG-538-M09_NODE_28_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-562-C07_contigs_AH-562-C07_NODE_38_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-562-E04_contigs_AH-562-E04_NODE_23_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-888-G20_contigs_AH-888-G20_NODE_29_2_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-121-N18_contigs_AM-121-N18_NODE_5_23_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-286-F05_contigs_AM-286-F05_NODE_1_41_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-562-O14_contigs_AH-562-O14_NODE_25_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-130-I22_contigs_AM-130-I22_NODE_14_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-201-F22_contigs_AM-201-F22_NODE_25_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-138-G09_contigs_AM-138-G09_NODE_34_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-562-K21_contigs_AH-562-K21_NODE_7_20_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-O03_contigs_AM-334-O03_NODE_1_55_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-888-A19_contigs_AH-888-A19_NODE_3_15_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-124-K09_contigs_AM-124-K09_NODE_2_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-288-O07_contigs_AM-288-O07_NODE_2_37_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-216-O05_contigs_AM-216-O05_NODE_7_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-735-G19_contigs_AH-735-G19_NODE_1_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-138-K04_contigs_AM-138-K04_NODE_10_19_rank:_C_sensory_rhodopsin", "AH-470-B04_contigs_AH-470-B04_NODE_24_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-K09_contigs_AH-470-K09_NODE_24_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-C23_contigs_AH-985-C23_NODE_7_17_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-M06_contigs_AH-473-M06_NODE_3_47_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-N17_contigs_AM-235-N17_NODE_10_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-B06_contigs_AH-470-B06_NODE_11_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-M19_contigs_AH-470-M19_NODE_23_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-N10_contigs_AH-470-N10_NODE_1_47_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-I08_contigs_AH-985-I08_NODE_14_11_rank:_C_sensory_rhodopsin", "AH-985-C14_contigs_AH-985-C14_NODE_2_20_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-C18_contigs_AM-235-C18_NODE_4_45_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-A20_contigs_AH-613-A20_NODE_8_12_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-982-B09_contigs_AH-982-B09_NODE_3_39_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-N06_contigs_AH-985-N06_NODE_7_54_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-276-D04_contigs_AM-276-D04_NODE_1_24_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-276-J16_contigs_AM-276-J16_NODE_7_12_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-685-M06_contigs_SCGC_AM-685-M06_contig16_8_rank:_C_sensory_rhodopsin", "AH-473-N15_contigs_AH-473-N15_NODE_8_38_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-473-F10_contigs_AH-473-F10_NODE_5_13_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-982-G18_contigs_AH-982-G18_NODE_6_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-G10_contigs_AH-985-G10_NODE_9_10_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-G13_contigs_AH-470-G13_NODE_9_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-264-E22_contigs_AM-264-E22_NODE_7_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-247-L05_contigs_AM-247-L05_NODE_22_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-K11_contigs_AM-187-K11_NODE_2_14_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-264-C11_contigs_AM-264-C11_NODE_1_39_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-685-I08_contigs_SCGC_AM-685-I08_contig20_7_rank:_C_sensory_rhodopsin", "AM-235-N11_contigs_AM-235-N11_NODE_31_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-118-N09_contigs_AM-118-N09_NODE_1_7_rank:_D_Bacteriorhodopsin-like", "AM-208-N16_contigs_AM-208-N16_NODE_4_13_rank:_D_Bacteriorhodopsin-like", "AM-118-L17_contigs_AM-118-L17_NODE_26_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-262-E04_contigs_AM-262-E04_NODE_12_4_rank:_D_Bacteriorhodopsin-like", "AM-334-F10_contigs_AM-334-F10_NODE_4_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-K03_contigs_AM-334-K03_NODE_11_11_rank:_C_sensory_rhodopsin", "AH-470-G14_contigs_AH-470-G14_NODE_2_14_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-P20_contigs_AH-470-P20_NODE_3_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-C03_contigs_AM-235-C03_NODE_1_35_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-A14_contigs_AH-613-A14_NODE_2_29_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-216-D21_contigs_AM-216-D21_NODE_4_68_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-P09_contigs_AM-235-P09_NODE_11_10_rank:_C_sensory_rhodopsin", "AM-235-E10_contigs_AM-235-E10_NODE_15_8_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-O04_contigs_AM-235-O04_NODE_13_11_rank:_C_sensory_rhodopsin", "AM-235-M03_contigs_AM-235-M03_NODE_9_19_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-E23_contigs_AM-235-E23_NODE_11_10_rank:_C_sensory_rhodopsin", "AM-229-I11_contigs_AM-229-I11_NODE_20_11_rank:_D_Bacteriorhodopsin-like", "AH-611-I17_contigs_AH-611-I17_NODE_27_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-K14_contigs_AH-613-K14_NODE_13_13_rank:_C_sensory_rhodopsin", "AM-247-J19_contigs_AM-247-J19_NODE_7_14_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-I21_contigs_AM-187-I21_NODE_16_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-138-L20_contigs_AM-138-L20_NODE_28_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-286-F19_contigs_AM-286-F19_NODE_1_21_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-262-D11_contigs_AM-262-D11_NODE_1_35_rank:_D_Bacteriorhodopsin-like", "AM-247-J13_contigs_AM-247-J13_NODE_5_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-225-P05_contigs_AM-225-P05_NODE_3_16_rank:_D_Bacteriorhodopsin-like", "AH-735-P08_contigs_AH-735-P08_NODE_30_1_rank:_D_Bacteriorhodopsin-like", "AH-735-M10_contigs_AH-735-M10_NODE_14_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-573-P17_contigs_AH-573-P17_NODE_5_40_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-187-A17_contigs_AM-187-A17_NODE_48_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-201-B23_contigs_AM-201-B23_NODE_1_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-229-G13_contigs_AM-229-G13_NODE_3_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-L02_contigs_AH-613-L02_NODE_15_10_rank:_C_sensory_rhodopsin", "AH-985-K15_contigs_AH-985-K15_NODE_10_23_rank:_C_sensory_rhodopsin", "AM-206-G06_contigs_AM-206-G06_NODE_20_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-288-C17_contigs_AM-288-C17_NODE_5_12_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-164-O15_contigs_AM-164-O15_NODE_25_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-262-M09_contigs_AM-262-M09_NODE_8_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-O03_contigs_AM-235-O03_NODE_1_61_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-201-O10_contigs_AM-201-O10_NODE_1_47_rank:_D_Bacteriorhodopsin-like", "AH-305-K19_contigs_AH-305-K19_NODE_3_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-573-A05_contigs_AH-573-A05_NODE_5_38_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-206-E11_contigs_AM-206-E11_NODE_2_25_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-264-E15_contigs_AM-264-E15_NODE_53_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-184-P01_contigs_AM-184-P01_NODE_4_30_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-184-P10_contigs_AM-184-P10_NODE_19_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-562-B18_contigs_AH-562-B18_NODE_9_23_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-255-F10_contigs_AM-255-F10_NODE_46_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-138-K02_contigs_AM-138-K02_NODE_3_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-K17_contigs_AM-334-K17_NODE_6_4_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-249-J22_contigs_AM-249-J22_NODE_1_25_rank:_D_Bacteriorhodopsin-like", "AH-470-K16_contigs_AH-470-K16_NODE_8_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-B06_contigs_AH-985-B06_NODE_2_77_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-288-K14_contigs_AM-288-K14_NODE_2_27_rank:_D_Bacteriorhodopsin-like", "AM-225-J05_contigs_AM-225-J05_NODE_12_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-247-K04_contigs_AM-247-K04_NODE_16_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-225-D02_contigs_AM-225-D02_NODE_16_2_rank:_D_Bacteriorhodopsin-like", "2L6X_1|Chain_A|Green-light_absorbing_proteorhodopsin|uncultured_marine_gamma_proteobacterium_EBAC31A08_(133804)", "AH-982-K14_contigs_AH-982-K14_NODE_2_13_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-982-L14_contigs_AH-982-L14_NODE_24_7_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-D10_contigs_AH-985-D10_NODE_1_25_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-I04_contigs_AH-985-I04_NODE_3_29_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-249-G03_contigs_AM-249-G03_NODE_15_11_rank:_C_sensory_rhodopsin", "AM-332-K07_contigs_AM-332-K07_NODE_5_45_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-N10_contigs_AM-332-N10_NODE_1_26_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-P19_contigs_AH-470-P19_NODE_7_12_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-991-O04_contigs_AH-991-O04_NODE_5_33_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-B08_contigs_AH-985-B08_NODE_6_26_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-985-G03_contigs_AH-985-G03_NODE_2_140_rank:_C_sensory_rhodopsin", "AH-985-A03_contigs_AH-985-A03_NODE_16_17_rank:_C_sensory_rhodopsin", "AH-985-G05_contigs_AH-985-G05_NODE_5_27_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-C04_contigs_AM-235-C04_NODE_1_95_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-E08_contigs_AM-235-E08_NODE_4_51_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-I10_contigs_AM-235-I10_NODE_12_24_rank:_C_sensory_rhodopsin", "AM-235-E05_contigs_AM-235-E05_NODE_2_5_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-J17_contigs_AM-235-J17_NODE_3_39_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-225-M23_contigs_AM-225-M23_NODE_1_72_rank:_D_Bacteriorhodopsin-like", "AM-262-B11_contigs_AM-262-B11_NODE_3_46_rank:_D_Bacteriorhodopsin-like", "AM-247-K13_contigs_AM-247-K13_NODE_9_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-201-N07_contigs_AM-201-N07_NODE_2_16_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-216-D03_contigs_AM-216-D03_NODE_26_13_rank:_C_sensory_rhodopsin", "AH-302-M20_contigs_AH-302-M20_NODE_2_51_rank:_D_Bacteriorhodopsin-like", "AM-255-G15_contigs_AM-255-G15_NODE_22_5_rank:_D_Bacteriorhodopsin-like", "AH-888-F10_contigs_AH-888-F10_NODE_20_5_rank:_D_Bacteriorhodopsin-like", "AM-208-B20_contigs_AM-208-B20_NODE_36_1_rank:_D_Bacteriorhodopsin-like", "AM-247-L03_contigs_AM-247-L03_NODE_2_27_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-991-N14_contigs_AH-991-N14_NODE_4_25_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-G02_contigs_AM-334-G02_NODE_15_11_rank:_D_Bacteriorhodopsin-like", "AM-276-J22_contigs_AM-276-J22_NODE_7_9_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-613-D20_contigs_AH-613-D20_NODE_11_9_rank:_D_Bacteriorhodopsin-like", "AM-235-E02_contigs_AM-235-E02_NODE_2_8_rank:_D_Bacteriorhodopsin-like", "AH-473-K22_contigs_AH-473-K22_NODE_11_9_rank:_D_Bacteriorhodopsin-like", "AH-473-P19_contigs_AH-473-P19_NODE_19_6_rank:_D_Bacteriorhodopsin-like", "AH-470-K14_contigs_AH-470-K14_NODE_10_17_rank:_D_Heliorhodopsin_PF18761.4", "AH-611-L08_contigs_AH-611-L08_NODE_22_11_rank:_D_Heliorhodopsin_PF18761.4", "AM-216-E19_contigs_AM-216-E19_NODE_2_58_rank:_D_Heliorhodopsin_PF18761.4", "AH-611-C09_contigs_AH-611-C09_NODE_33_4_rank:_D_Heliorhodopsin_PF18761.4", "AH-613-E18_contigs_AH-613-E18_NODE_21_10_rank:_D_Heliorhodopsin_PF18761.4", "AH-611-N13_contigs_AH-611-N13_NODE_4_83_rank:_D_Heliorhodopsin_PF18761.4", "AM-216-O15_contigs_AM-216-O15_NODE_69_2_rank:_D_Heliorhodopsin_PF18761.4", "AM-332-K22_contigs_AM-332-K22_NODE_84_3_rank:_D_Heliorhodopsin_PF18761.4", "AM-334-L09_contigs_AM-334-L09_NODE_11_13_rank:_D_Heliorhodopsin_PF18761.4", "AM-390-K04_contigs_AM-390-K04_NODE_144_1_rank:_D_Heliorhodopsin_PF18761.4", "AM-390-N10_contigs_AM-390-N10_NODE_37_2_rank:_D_Heliorhodopsin_PF18761.4", "AM-390-N10_contigs_AM-390-N10_NODE_11_22_rank:_D_Heliorhodopsin_PF18761.4", "AM-685-I13_contigs_SCGC_AM-685-I13_contig155_7_rank:_D_Heliorhodopsin", "AM-388-E15_contigs_AM-388-E15_NODE_18_8_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-E13_contigs_AM-388-E13_NODE_96_4_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-E13_contigs_AM-388-E13_NODE_41_12_rank:_D_Heliorhodopsin_PF18761.4", "AM-689-G20_contigs_SCGC_AM-689-G20_contig37_7_rank:_D_Heliorhodopsin", "AM-685-N21_contigs_SCGC_AM-685-N21_contig3_18_rank:_D_Heliorhodopsin", "AH-470-P09_contigs_AH-470-P09_NODE_19_5_rank:_D_Heliorhodopsin_PF18761.4", "AH-613-J13_contigs_AH-613-J13_NODE_27_2_rank:_D_Heliorhodopsin_PF18761.4", "AH-613-J13_contigs_AH-613-J13_NODE_27_8_rank:_D_Heliorhodopsin_PF18761.4", "AH-982-J08_contigs_AH-982-J08_NODE_5_2_rank:_D_Heliorhodopsin_PF18761.4", "AH-613-M18_contigs_AH-613-M18_NODE_35_2_rank:_D_Heliorhodopsin_PF18761.4", "AM-235-A19_contigs_AM-235-A19_NODE_30_2_rank:_D_Heliorhodopsin_PF18761.4", "AH-613-M18_contigs_AH-613-M18_NODE_35_5_rank:_D_Heliorhodopsin_PF18761.4", "AH-613-O14_contigs_AH-613-O14_NODE_1_17_rank:_D_Heliorhodopsin_PF18761.4", "AM-229-B10_contigs_AM-229-B10_NODE_1_16_rank:_D_Heliorhodopsin_PF18761.4", "AM-229-F05_contigs_AM-229-F05_NODE_1_93_rank:_D_Heliorhodopsin_PF18761.4", "AM-243-K20_contigs_AM-243-K20_NODE_12_8_rank:_D_Heliorhodopsin_PF18761.4", "AM-235-C15_contigs_AM-235-C15_NODE_12_27_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-F07_contigs_AM-388-F07_NODE_9_33_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-I04_contigs_AM-388-I04_NODE_5_24_rank:_D_Heliorhodopsin_PF18761.4", "AH-611-J11_contigs_AH-611-J11_NODE_12_6_rank:_D_Heliorhodopsin_PF18761.4", "AH-611-N17_contigs_AH-611-N17_NODE_13_1_rank:_D_Heliorhodopsin_PF18761.4", "AH-611-N17_contigs_AH-611-N17_NODE_26_8_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-D06_contigs_AM-388-D06_NODE_9_3_rank:_D_Heliorhodopsin_PF18761.4", "AM-390-K03_contigs_AM-390-K03_NODE_6_42_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-N06_contigs_AM-388-N06_NODE_5_21_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-O03_contigs_AM-388-O03_NODE_2_60_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-O19_contigs_AM-388-O19_NODE_5_66_rank:_D_Heliorhodopsin_PF18761.4", "AM-390-G11_contigs_AM-390-G11_NODE_3_42_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-G18_contigs_AM-388-G18_NODE_7_28_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-I02_contigs_AM-388-I02_NODE_4_30_rank:_D_Heliorhodopsin_PF18761.4", "AM-390-I08_contigs_AM-390-I08_NODE_17_2_rank:_D_Heliorhodopsin_PF18761.4", "AM-390-L11_contigs_AM-390-L11_NODE_31_7_rank:_D_Heliorhodopsin_PF18761.4", "AM-201-P14_contigs_AM-201-P14_NODE_2_22_rank:_D_Heliorhodopsin_PF18761.4", "AM-276-F17_contigs_AM-276-F17_NODE_3_56_rank:_D_Heliorhodopsin_PF18761.4", "AM-388-D13_contigs_AM-388-D13_NODE_7_8_rank:_D_Heliorhodopsin_PF18761.4", "AM-235-L17_contigs_AM-235-L17_NODE_31_5_rank:_D_Heliorhodopsin_PF18761.4", "AH-611-J11_contigs_AH-611-J11_NODE_3_27_rank:_D_Heliorhodopsin_PF18761.4", "AH-473-C06_contigs_AH-473-C06_NODE_1_137_rank:_D_Bacteriorhodopsin-like", "AH-982-A16_contigs_AH-982-A16_NODE_15_2_rank:_D_Bacteriorhodopsin-like", "AH-985-F16_contigs_AH-985-F16_NODE_3_65_rank:_D_Bacteriorhodopsin-like", "AM-235-L04_contigs_AM-235-L04_NODE_71_3_rank:_D_Bacteriorhodopsin-like", "AM-388-E22_contigs_AM-388-E22_NODE_3_24_rank:_D_Bacteriorhodopsin-like", "AM-388-M09_contigs_AM-388-M09_NODE_4_31_rank:_D_Bacteriorhodopsin-like", "AM-390-O23_contigs_AM-390-O23_NODE_3_93_rank:_D_Bacteriorhodopsin-like", "AM-388-O19_contigs_AM-388-O19_NODE_8_36_rank:_D_Bacteriorhodopsin-like", "AM-390-A03_contigs_AM-390-A03_NODE_19_17_rank:_D_Bacteriorhodopsin-like", "AM-390-G11_contigs_AM-390-G11_NODE_49_5_rank:_D_Bacteriorhodopsin-like", "AM-390-M10_contigs_AM-390-M10_NODE_8_35_rank:_D_Bacteriorhodopsin-like", "AM-390-G15_contigs_AM-390-G15_NODE_14_9_rank:_D_Bacteriorhodopsin-like", "AM-235-L22_contigs_AM-235-L22_NODE_18_9_rank:_D_Bacteriorhodopsin-like", "AM-332-C18_contigs_AM-332-C18_NODE_18_2_rank:_D_Bacteriorhodopsin-like", "AM-390-L10_contigs_AM-390-L10_NODE_2_31_rank:_D_Bacteriorhodopsin-like", "AM-332-I17_contigs_AM-332-I17_NODE_1_24_rank:_D_Bacteriorhodopsin-like", "AM-158-K03_contigs_AM-158-K03_NODE_11_13_rank:_D_Bacteriorhodopsin-like", "AM-276-B16_contigs_AM-276-B16_NODE_99_1_rank:_D_Bacteriorhodopsin-like", "AM-201-P14_contigs_AM-201-P14_NODE_1_74_rank:_D_Bacteriorhodopsin-like", "AM-276-F17_contigs_AM-276-F17_NODE_7_19_rank:_D_Bacteriorhodopsin-like", "AM-235-E04_contigs_AM-235-E04_NODE_5_1_rank:_D_Bacteriorhodopsin-like", "AM-332-J08_contigs_AM-332-J08_NODE_48_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-L05_contigs_AM-332-L05_NODE_64_3_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-J02_contigs_AM-332-J02_NODE_17_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-243-M17_contigs_AM-243-M17_NODE_2_29_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-K04_contigs_AM-332-K04_NODE_7_39_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-D22_contigs_AM-332-D22_NODE_6_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-685-M09_contigs_SCGC_AM-685-M09_contig9_44_rank:_C_sensory_rhodopsin", "AM-235-D03_contigs_AM-235-D03_NODE_1_1_rank:_C_sensory_rhodopsin_(db=kegg)", "AH-470-O06_contigs_AH-470-O06_NODE_6_68_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-334-A08_contigs_AM-334-A08_NODE_34_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-235-J06_contigs_AM-235-J06_NODE_13_6_rank:_C_sensory_rhodopsin_(db=kegg)", "AM-332-O16_contigs_AM-332-O16_NODE_9_12_rank:_C_sensory_rhodopsin_(db=kegg)"]
print("Retrieving full names of those rhodopsins")

LIST_seqNames_to_keep = []
for longname in LIST_full_seqIDs:
    shortname = re.sub("_rank.*",'',longname).replace('_',' ')
    if shortname in LIST_non2Fe2S_to_check_for_8th_helix:
        LIST_seqNames_to_keep.append(longname)
print(str(len(LIST_seqNames_to_keep)) + ' non2Fe2S rhods to keep (other was a divergent rhodopsin that didnt meet inclusion criteria for the iToL tree)')

LIST_seqIDs_to_keep = []
for name in LIST_seqNames_to_keep:
    ID = re.sub("_rank.*",'',name)
    LIST_seqIDs_to_keep.append(ID)

#### A. Write out 2Fe2S rhodopsins to their own fasta
LIST_2Fe2S_records = []
for record in SeqIO.parse(PATH_input_1, "fasta"):
    if record.name.replace('_',' ').rstrip() in LIST_2Fe2S_rhodopsins:
        LIST_2Fe2S_records.append(record)
print(str(len(LIST_2Fe2S_records)) + " 2Fe2S_rhodopsins to write to fasta:\n\t"+PATH_output_1a)
# Write it out
with open(PATH_output_1a, "w") as output_handle:
    SeqIO.write(LIST_2Fe2S_records, output_handle, "fasta")

#### B. Write out non2Fe2S rhodopsins to a separate fasta
count=0
LIST_other_records=[]
for record in SeqIO.parse(PATH_input_1, "fasta"):
    if record.name in LIST_seqIDs_to_keep:
        count+=1
        LIST_other_records.append(record)
print(str(count))
# Write it out
with open(PATH_output_1b, "w") as output_handle:
    SeqIO.write(LIST_other_records, output_handle, "fasta")
print("Writing out alignment file to find non-2Fe2S rhodopsins to visually check (in alphafold) for presence of an 8th helix.\n\tTo: "+PATH_output_1b)
print("Done.")

OVERVIEW

I have visually inspected the alignment ./0_input_rhod_alignment_by_Tianyi/aligned_sequences.fasta
And found 603 GORG Dark rhodopsins that have alignable AAs possibly forming an N-terminal 8th helix
Part I of this notebook will subset that alignment into two output fastas containing.
	(1)The 2Fe2S rhodopsins
	(2)The Non2Fe2S rhodopsins
For further visual analysis. I.e. So that I can group them by seq similarity and then visually evaluate one rep of each group in alphafold.



PART I

461 PROTEOrhodopsins phylogenetically branching within 2Fe2S rhodopsins
Retrieving full names of those rhodopsins
141 non2Fe2S rhods to keep (other was a divergent rhodopsin that didnt meet inclusion criteria for the iToL tree)
461 2Fe2S_rhodopsins to write to fasta:
	./1_output_simplified_alignment_for_rhods_to_visually_screen/461_2Fe2S_rhodopsins_that_might_have_helix.fasta
141
Writing out alignment file to find non-2Fe2S rhodopsins to visually check (in alphafold) for presence of an 8th helix.

In [2]:
######## PART II. ####### 
####
#### CONTEXT:
#### Having visually screened those non-2Fe2S rhodopsins in alphafold
####     I identified the 67 rhods with an 8th helix
####
#### GOAL: Combine the list of ALL RHODS WITH SEEMINGLY COMPLETE 8th HELIX (i.e. at least two full rotations)
####  (i.e. 427 complete 2Fe2Srhods + 67 others -> 494 rhods)
####
#### PRODUCT: A subset of the original rhodopsin AA alignment file.
####  (1065 -> 494 rhods, with gaps still)
####
#### POST-processing: Tianyi should re-align these AAs
####   to make an HMM profile of the 8th helix

DIR_input_2 = "./2_input_rhod_lists_based_on_visual_screening/"
PATH_input_2a = DIR_input_2 + "67_rhods_approved_visually_in_alphafold.txt"
PATH_input_2b = DIR_input_2 + "16_2Fe2S_rhods_incomplete_helix.txt"
PATH_input_2c = DIR_input_2 + "18_2Fe2S_rhods_missing_helix.txt"

PATH_output_2a = "./3_output_rhod_for_Tianyi_to_realign/blah"

with open(PATH_input_2a) as file: LIST_67_rhod = [line.rstrip() for line in file]    ## Read in list of rhods that have 8th helix 
with open(PATH_input_2b) as file: LIST_rhods_w_incomplete_helix = [line.rstrip() for line in file]    ## Read in list of rhods that have 8th helix 
with open(PATH_input_2c) as file: LIST_rhods_missing_helix = [line.rstrip() for line in file]    ## Read in list of rhods that have 8th helix 

LIST_final_IDs = []
LIST_final_records = []

LIST_2Fe2S_with_helix_regardless_of_completeness = [ i for i in LIST_2Fe2S_rhodopsins if i not in LIST_rhods_missing_helix ]
LIST_2Fe2S_with_complete_helix = [ i for i in LIST_2Fe2S_rhodopsins if i not in LIST_rhods_missing_helix and i not in LIST_rhods_w_incomplete_helix ]

_LIST_has_helix_regardless_of_completeness = LIST_67_rhod + LIST_2Fe2S_with_helix_regardless_of_completeness
LIST_has_helix_regardless_of_completeness = []
for name in _LIST_has_helix_regardless_of_completeness:
    if '_' in name:
        name = re.sub("_rank.*",'',name).replace('_',' ') ## Adjust name format to match alignment file
    LIST_has_helix_regardless_of_completeness.append(name)

_LIST_rhods_with_complete_helix = LIST_2Fe2S_with_complete_helix + LIST_67_rhod
LIST_rhods_with_complete_helix = []
for name in _LIST_rhods_with_complete_helix:
    if '_' in name:
        name = re.sub("_rank.*",'',name).replace('_',' ') ## Adjust name format to match alignment file
    LIST_rhods_with_complete_helix.append(name)


print("STATS...")
print("Based on my visual review of the 3D alphafold models")
print(str(len(LIST_67_rhod)) + " non2Fe2S rhods have 8th helix")
print("Of "+str(len(LIST_2Fe2S_rhodopsins))+" 2Fe2S rhods...")
print("\t"+str(len(LIST_rhods_missing_helix))+" are missing the 8th helix")
print("\t"+str(len(LIST_rhods_w_incomplete_helix))+" have an incomplete one.")
print("Therefore, in total "+str(len(LIST_rhods_with_complete_helix))+" rhods have a seemingly complete 8th helix")
print("While "+str(len(LIST_has_helix_regardless_of_completeness))+" rhods have an 8th helix in some capacity")

print("\nPROCESSING...")
print("Saving desired subset to fasta...")
#### Vary logic to get the subet you want
for name in LIST_2Fe2S_rhodopsins:
    if ( name not in LIST_rhods_w_incomplete_helix and name not in LIST_rhods_missing_helix):
        ID = name.replace(' ','_') ## Adjust name format to match alignment file
        LIST_final_IDs.append(ID)

for name in LIST_67_rhod:
   ID = re.sub("_rank.*",'',name) ## Adjust name format to match alignment file
   LIST_final_IDs.append(ID)

#### Extract those from Tianyi's initial alignment
for record in SeqIO.parse("./0_input_rhod_alignment_by_Tianyi/aligned_sequences.fasta", "fasta"):
    if record.id in LIST_final_IDs:
        LIST_final_records.append(record)
        
print("= " + str(len(LIST_final_records)) + ' Proteins retrieved')

print("\nPublishing those to a new fasta alignment file (Needs to be realigned)")

# Write it out
with open(PATH_output_2a, "w") as output_handle:
    SeqIO.write(LIST_final_records, output_handle, "fasta")

print("Done!")

STATS...
Based on my visual review of the 3D alphafold models
67 non2Fe2S rhods have 8th helix
Of 461 2Fe2S rhods...
	18 are missing the 8th helix
	16 have an incomplete one.
Therefore, in total 494 rhods have a seemingly complete 8th helix
While 510 rhods have an 8th helix in some capacity

PROCESSING...
Saving desired subset to fasta...
= 494 Proteins retrieved

Publishing those to a new fasta alignment file (Needs to be realigned)
Done!


In [39]:
len(LIST_has_helix_regardless_of_completeness)

510

In [6]:
######## PART III. ####### 

#### A. Update iToL table with "8th helix" column (presence, absence)

#### B. Update rhodopsin table with new columns:
# '2Fe2S_clade' (True/False)
# 8th_helix_alphafold (True/False)
# 8th_helix_incomplete (True/False)

# PATH_to_rhodopsin_metadata_table
DIR_to_look_for_iToL_table = "../6_most_common_operons/output/"

from glob import glob
import sys
import os
import pandas as pd
import numpy as np

####### A. UPDATE THE ITOL TABLE ########
LIST_iToL_tables = glob(DIR_to_look_for_iToL_table+"*csv")
print("Found " + str(len(LIST_iToL_tables)) + " tables for iToL that I can update with helix info.")
if len(LIST_iToL_tables) != 1:
    sys.error("ERROR. Expecting exactly one iToL table but found "+str(len(LIST_iToL_tables))+".\nUnclear which one to update.")

PATH_iToL_input = LIST_iToL_tables[0]
NAME_iToL_output = os.path.basename(PATH_iToL_input).replace('.csv','_plus_8thHelix.csv')
DF = pd.read_csv(PATH_iToL_input)

# Replace spaces with underscores to match iToL SeQID formatting
LIST_has_helix_regardless_of_completeness = [ i.replace(' ','_') for i in LIST_has_helix_regardless_of_completeness]

### Check that SeqIDs map to iToL rhodopsin names
count_matched = 0
for ID in LIST_has_helix_regardless_of_completeness:
    if ID in DF['rhodopsin_gene'].tolist():
        count_matched+=1
print(str(count_matched) + " out of " + str(len(LIST_has_helix_regardless_of_completeness)) + " were mapped to rhodopsin SeqIDs in the iToL table")
if count_matched != len(LIST_has_helix_regardless_of_completeness):
    sys.exit("ERROR. Not all SeqIDs match iToL rhodopsin names.")

DICT_helix_present_absent = {}
for ID in DF['rhodopsin_gene'].tolist():
    # Helix present
    if ID in LIST_has_helix_regardless_of_completeness:
        DICT_helix_present_absent[ID]=1
    # Helix absent
    else:
        DICT_helix_present_absent[ID]=0

### Finally, add the new column
DF['8th helix'] = DF['rhodopsin_gene'].map(DICT_helix_present_absent)
DF['8th helix'].value_counts()

### Save the results
print("Saving iToL table")
DF.to_csv(NAME_iToL_output)

####### B. UPDATE THE RHODOPSIN METADATA TABLE ########

print("Done")

Found 1 tables for iToL that I can update with helix info.
510 out of 510 were mapped to rhodopsin SeqIDs in the iToL table
Saving iToL table
Done


In [5]:
LIST_iToL_tables

[]

In [8]:
######## PART IV. ####### 
#### Update rhodopsin metadata table with new columns:
# 'clade' (True/False)
# 8th_helix_alphafold (True/False)
# 8th_helix_incomplete (True/False)

PATH_output_4 = "GORG_dark_rhodopsin_metadata_with_phylogeny_and_alphafold3.tsv"

### INPUTS #####

# Metadata about rhodopsins
# A.k.a. spectral tuning, neighboring genes, etc
PATH_rhodopsin_metadata = "./0_input_rhodopsin_metadata_by_Tianyi/rhodopsin_type_neighborhood.csv"
DF_meta = pd.read_csv(PATH_rhodopsin_metadata)
print(str(len(DF_meta)) + ' rhodopsins in DF_meta')

# Rhodopsin phylogeny
# File obtained from Maria Pachiadaki, was originally named 'Rhodopsin_genes_Dark_info.txt'
PATH_rhodopsin_phylogeny = "./4_input_rhodopsin_metadata/rhodopsin_phylogeny.tsv"
DF_phylo = pd.read_csv(PATH_rhodopsin_phylogeny, sep="\t")
DF_phylo.rename(columns={'Gene_id':'rhodopsin_gene','Family':'rhodopsin_family'}, inplace=True)
DF_phylo.drop(columns=['SAG','Notes'],inplace=True)
print(str(len(DF_phylo)) + ' rhodopsins in DF_phylo')
print("^ Fewer rhodopsins in phylogeny than in metadata, because not all rhodopsins could be aligned and were thus excluded as aberrant/incomplete.\n")


#### MERGE TABLES ######

print("Merging tables:")
DF_out = DF_meta.merge(DF_phylo, on="rhodopsin_gene", how="left")
print(str(len(DF_out)) + " rows after merge.\n" )



##### ADD NEW COLUMNS #####

print("Adding new columns:")

### 'rhodopsin_subfamily' column
DF_out['rhodopsin_subfamily'] = ''
DF_out['rhodopsin_subfamily'] =  np.where(DF_out['rhodopsin_gene'].str.replace('_',' ').isin(LIST_2Fe2S_rhodopsins), '2Fe2S-associated rhodopsin', DF_out['rhodopsin_subfamily'])
print(str(len(LIST_2Fe2S_rhodopsins)) + ' rhodopsins were in the 2Fe2S subfamily of proteorhodopsins')
DF_out['rhodopsin_subfamily'] = np.where(DF_out['rhodopsin_gene'].str.replace('_',' ').isin(LIST_FA_rhodopsins), 'flotillin-associated rhodopsin', DF_out['rhodopsin_subfamily'])
print(str(len(LIST_FA_rhodopsins)) + ' rhodopsins were in the flotillin-associated subfamily of proteorhodopsins')

#### 'suitable_for_phylogenetics' column (True/False)
LIST_rhods_suitable_for_phylogeny = DF_phylo['rhodopsin_gene'].tolist()
DF_out['suitable_for_phylogenetics'] = np.where(DF_out['rhodopsin_gene'].isin(LIST_rhods_suitable_for_phylogeny),True,False)
print(str(len(LIST_rhods_suitable_for_phylogeny)) + ' rhodopsins were suitable for phylogenetics')

####  '8th_N-terminal_helix' column (True/False)
DF_out['8th_N-terminal_helix (alphafold3)'] = np.where(DF_out['rhodopsin_gene'].str.replace('_',' ').isin(LIST_rhods_with_complete_helix),True,False)
print(str(len(LIST_rhods_with_complete_helix)) + ' rhodopsins have an 8th helix (based on alphafold3)')

#### 'incomplete_8th_helix' column (True/False)
DF_out['incomplete_8th_helix (alphafold3)'] = np.where(DF_out['rhodopsin_gene'].str.replace('_',' ').isin(LIST_rhods_w_incomplete_helix),True,False)
print(str(len(LIST_rhods_w_incomplete_helix)) + ' rhodopsins have an INCOMPLETE 8th helix')


##### FINISH AND SAVE #######
DF_out.to_csv(PATH_output_4, sep="\t")

1093 rhodopsins in DF_meta
1052 rhodopsins in DF_phylo
^ Fewer rhodopsins in phylogeny than in metadata, because not all rhodopsins could be aligned and were thus excluded as aberrant/incomplete.

Merging tables:
1093 rows after merge.

Adding new columns:
461 rhodopsins were in the 2Fe2S subfamily of proteorhodopsins
395 rhodopsins were in the flotillin-associated subfamily of proteorhodopsins
1052 rhodopsins were suitable for phylogenetics
494 rhodopsins have an 8th helix (based on alphafold3)
16 rhodopsins have an INCOMPLETE 8th helix


In [9]:
DF_out

,rhodopsin_gene,rhodopsin_scaffold,sag,rhodopsin_count_in_sag,carotenoid_gene_cluster,flotillin,neither,flanking_genes,color_tuning_type,color_tuning_residue,three_residue_motif,pump_type,gtdb,niche,rhodopsin_family,rhodopsin_subfamily,suitable_for_phylogenetics,8th_N-terminal_helix (alphafold3),incomplete_8th_helix (alphafold3)
0,AG-538-A02_contigs_AG-538-A02_NODE_4_9,AG-538-A02_NODE_4,AG-538-A02,1,0,1,0,{'pfam_hits of gene #-5': ['Domain of unknown ...,BPR,Q,DTT,Other,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,MES,Proteorhodopsins,flotillin-associated rhodopsin,True,False,False
1,AG-538-D18_contigs_AG-538-D18_NODE_2_12,AG-538-D18_NODE_2,AG-538-D18,1,0,1,0,"{'pfam_hits of gene #-5': ['Histidine kinase-,...",BPR,Q,DTL,Other,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,MES,Proteorhodopsins,flotillin-associated rhodopsin,True,False,False
2,AG-538-B02_contigs_AG-538-B02_NODE_14_4,AG-538-B02_NODE_14,AG-538-B02,1,0,0,0,{'pfam_hits of gene #-3': ['Amidohydrolase fam...,BPR,Q,ETL,Other,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,MES,Proteorhodopsins,2Fe2S-associated rhodopsin,True,True,False
3,AG-538-C05_contigs_AG-538-C05_NODE_1_19,AG-538-C05_NODE_1,AG-538-C05,1,0,1,0,{'pfam_hits of gene #-5': ['Periplasmic bindin...,BPR,Q,DTT,Other,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,MES,Proteorhodopsins,flotillin-associated rhodopsin,True,False,False
4,AG-538-B18_contigs_AG-538-B18_NODE_3_16,AG-538-B18_NODE_3,AG-538-B18,1,0,1,0,{'pfam_hits of gene #-5': ['Glutamine amidotra...,Other,T,DTT,Other,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,MES,Proteorhodopsins,flotillin-associated rhodopsin,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,AM-689-G20_contigs_SCGC_AM-689-G20_contig37_7,SCGC_AM-689-G20_contig37,AM-689-G20,1,0,0,0,{'pfam_hits of gene #-5': ['4TM region of pyri...,Other,M,ESI,Other,d__Archaea;p__Thermoplasmatota;c__Poseidoniia;...,Black,NaN,,False,False,False
1089,AM-689-N03_contigs_SCGC_AM-689-N03_contig1_25,SCGC_AM-689-N03_contig1,AM-689-N03,2,0,0,1,"{'pfam_hits of gene #-5': [""'Cold-shock' DNA-b...",BPR,Q,DTE,proton,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,Black,Proteorhodopsins,2Fe2S-associated rhodopsin,True,True,False
1090,AM-685-N21_contigs_SCGC_AM-685-N21_contig3_18,SCGC_AM-685-N21_contig3,AM-685-N21,1,0,0,1,{'pfam_hits of gene #-5': ['NUBPL iron-transfe...,Other,M,ESA,Other,d__Archaea;p__Thermoplasmatota;c__EX4484-6;o__...,Black,Heliorhodopsins,,True,False,False
1091,AM-689-M16_contigs_SCGC_AM-689-M16_contig19_3,SCGC_AM-689-M16_contig19,AM-689-M16,1,0,0,0,"{'pfam_hits of gene #-2': ['None', 'None', 3, ...",Other,M,ESI,Other,d__Bacteria;p__Patescibacteria;c__ABY1;o__UM-F...,Black,NaN,,False,False,False


In [39]:
LIST_rhods_with_complete_helix

['AM-332-B10 contigs AM-332-B10 NODE 8 9',
 'AM-332-A16 contigs AM-332-A16 NODE 16 2',
 'AM-334-C06 contigs AM-334-C06 NODE 11 9',
 'AM-332-L17 contigs AM-332-L17 NODE 8 10',
 'AM-332-N13 contigs AM-332-N13 NODE 3 11',
 'AM-235-B08 contigs AM-235-B08 NODE 6 15',
 'AM-332-N20 contigs AM-332-N20 NODE 1 20',
 'AM-332-B18 contigs AM-332-B18 NODE 12 2',
 'AM-259-O02 contigs AM-259-O02 NODE 1 31',
 'AM-130-G06 contigs AM-130-G06 NODE 6 65',
 'AH-735-N13 contigs AH-735-N13 NODE 29 5',
 'AM-247-J07 contigs AM-247-J07 NODE 30 6',
 'AM-164-N21 contigs AM-164-N21 NODE 43 6',
 'AM-288-P17 contigs AM-288-P17 NODE 2 39',
 'AH-985-A17 contigs AH-985-A17 NODE 1 74',
 'AH-470-P15 contigs AH-470-P15 NODE 3 1',
 'AM-332-F18 contigs AM-332-F18 NODE 2 10',
 'AM-332-C06 contigs AM-332-C06 NODE 4 4',
 'AM-288-B03 contigs AM-288-B03 NODE 7 10',
 'AM-130-K09 contigs AM-130-K09 NODE 64 1',
 'AM-332-A22 contigs AM-332-A22 NODE 1 15',
 'AM-332-N08 contigs AM-332-N08 NODE 21 5',
 'AM-334-N11 contigs AM-334-N11 NOD

In [11]:
DF_phylo.columns

Index(['Gene_id', 'Family', 'SAG', 'Notes'], dtype='object')

In [22]:
print("STATS...")
print("Based on my visual review of the 3D alphafold models")
print(str(len(LIST_67_rhod)) + " non2Fe2S rhods have 8th helix")
print("Of "+str(len(LIST_2Fe2S_rhodopsins))+" 2Fe2S rhods...")
print("\t"+str(len(LIST_rhods_missing_helix))+" are missing the 8th helix")
print("\t"+str(len(LIST_rhods_w_incomplete_helix))+" have an incomplete one.")
print("Therefore, in total "+str(len(LIST_rhods_with_complete_helix))+" rhods have a seemingly complete 8th helix")
print("While "+str(len(LIST_has_helix_regardless_of_completeness))+" rhods have an 8th helix in some capacity")

1093

In [5]:
DF.columns

Index(['rhodopsin_gene', '3' 2Fe-2S binding dom.', '3' flotillin',
       '5' AMP-binding enz.', '5' VirC1', '5' amidohydrolase fam.',
       '5' A.A. permease', '5' periplasmic-binding prot.', '3' CrtE',
       '3' L-rhamnose isomerase', '5' brp/blh', '3' tRNA synthetase I',
       '3' haemolysin-III related', '5' fasciclin', '3' fatty acid desaturase',
       '3' brp/blh', '5' thiamine pyrophosphate', '5' TMEM205-like',
       '5' AI-2E fam. transporter', '5' toxin-antitoxin',
       '3' phosphotransferase fam.', '3' SOUL heme-binding prot.',
       '3' ABC1 kinase-like', '5' FGGY fam. carb. kinases', '8th helix'],
      dtype='object')

In [7]:
DF_meta.head()

,rhodopsin_gene,rhodopsin_scaffold,sag,rhodopsin_count_in_sag,carotenoid_gene_cluster,flotillin,neither,flanking_genes,color_tuning_type,color_tuning_residue,three_residue_motif,pump_type,gtdb,niche
0,AG-538-A02_contigs_AG-538-A02_NODE_4_9,AG-538-A02_NODE_4,AG-538-A02,1,0,1,0,{'pfam_hits of gene #-5': ['Domain of unknown ...,BPR,Q,DTT,Other,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,MES
1,AG-538-D18_contigs_AG-538-D18_NODE_2_12,AG-538-D18_NODE_2,AG-538-D18,1,0,1,0,"{'pfam_hits of gene #-5': ['Histidine kinase-,...",BPR,Q,DTL,Other,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,MES
2,AG-538-B02_contigs_AG-538-B02_NODE_14_4,AG-538-B02_NODE_14,AG-538-B02,1,0,0,0,{'pfam_hits of gene #-3': ['Amidohydrolase fam...,BPR,Q,ETL,Other,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,MES
3,AG-538-C05_contigs_AG-538-C05_NODE_1_19,AG-538-C05_NODE_1,AG-538-C05,1,0,1,0,{'pfam_hits of gene #-5': ['Periplasmic bindin...,BPR,Q,DTT,Other,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,MES
4,AG-538-B18_contigs_AG-538-B18_NODE_3_16,AG-538-B18_NODE_3,AG-538-B18,1,0,1,0,{'pfam_hits of gene #-5': ['Glutamine amidotra...,Other,T,DTT,Other,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,MES


In [45]:
DF.columns

Index(['rhodopsin_gene', '3' 2Fe-2S binding dom.', '3' flotillin',
       '5' AMP-binding enz.', '5' VirC1', '5' amidohydrolase fam.',
       '5' A.A. permease', '5' periplasmic-binding prot.', '3' CrtE',
       '3' L-rhamnose isomerase', '5' brp/blh', '3' tRNA synthetase I',
       '3' haemolysin-III related', '5' fasciclin', '3' fatty acid desaturase',
       '3' brp/blh', '5' thiamine pyrophosphate', '5' TMEM205-like',
       '5' AI-2E fam. transporter', '5' toxin-antitoxin',
       '3' phosphotransferase fam.', '3' SOUL heme-binding prot.',
       '3' ABC1 kinase-like', '5' FGGY fam. carb. kinases'],
      dtype='object')

In [26]:
DF['rhodopsin_gene'].tolist()[0]

'AG-538-A02_contigs_AG-538-A02_NODE_4_9'

In [25]:
LIST_2Fe2S_with_helix_regardless_of_completeness[0]

'AM-332-B10_contigs_AM-332-B10_NODE_8_9'

In [29]:
'AM-332-B10_contigs_AM-332-B10_NODE_8_9' in DF['rhodopsin_gene'].tolist()

True

In [67]:
#### PART III ####
## Subset Tianyi's alignment to have only the 461 2Fe2Srhods
### Why? So I can align them and determine which of them ACTUALLY have the 8th helix (some will be truncated)


'AH-470-E07_contigs_AH-470-E07_NODE_2_54_rank:_C_sensory_rhodopsin_(db=kegg)'

In [104]:
print(str(len(LIST_2Fe2S_rhodopsins)) + ' 2Fe2S-type rhodopsin sequences in GORG-Dark')
print("But it isnt safe to assume they all have the 8th helix.\n\t(Since Greg has visually inspected only 40 of them in Alphafold.)")
print("Therefore, subsetting them from Tianyi's alignment of ALL ~1200 gDARK rhods.")
print("So that Greg can...\n\t(1) Re-align just the 2Fe2S rhodopsins\n\t(2) Thereby grouping them by sequence similarity, i.e. into alignment blocks, so that... \n\t(3) He can visually inspect representatives from each alignment block, in Alphafold3.")
print("\nULTIMATE GOAL: Get a list of which 2Fe2S rhods actually have the 8th helix.")

461 2Fe2S-type rhodopsin sequences in GORG-Dark
But it isnt safe to assume they all have the 8th helix.
	(Since Greg has visually inspected only 40 of them in Alphafold.)
Therefore, subsetting them from Tianyi's alignment of ALL ~1000 gDARK rhods.
So that Greg can...
	(1) Re-align just the 2Fe2S rhodopsins
	(2) Thereby grouping them by sequence similarity, i.e. into alignment blocks, so that... 
	(3) He can visually inspect representatives from each alignment block, in Alphafold3.

ULTIMATE GOAL: Get a list of which 2Fe2S rhods actually have the 8th helix.


461 2Fe2S_rhodopsins to write to fasta


In [117]:
val.replace(' ','_')

'AH-307-N20_contigs_AH-307-N20_NODE_19_15'

In [114]:
val.replace(' ','_')

'AH-307-N20_contigs_AH-307-N20_NODE_19_15'